In [ ]:
#
# Injectivity-o-matic
# - code workbook to generate injectivity indices, frac pressures etc. for all injectors
# - flags all periods of injection life as being bean-up, steady-state, shut-in etc. 
# - runs some cool algorithms to scrub P(Q) data and do (piecewise) linear regressions of these periods
#
# - this code workbook generates all of the underlying data sets; a separate code workbook / 
#     Spotfire file will be used to visualise the outputs
#
# Contact: D Robbins
#
# Global libraries
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, optimize, interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.linear_model import LinearRegression
#
# Useful global conversion factors
f_bar_to_psi = 14.5037743877082
f_m3hr_to_bpd = 150.95544977665
f_II_metric_to_field = f_m3hr_to_bpd/f_bar_to_psi
#
# Steady-state II sampling
i_sample_SS = 10   # number of rows for each steady-state II calculation to avoid huge data sets (should be equivalent to number of minutes)
#
# Bean-up II stampling
i_add_previous_row   = 0 # if fewer than this number of P(Q) points in a cluster, add previous PQ value for standard linear regression (0 = will always do this, recommended)
i_double_knots       = 0 # if 1, double up knot points for piecewise linear regression (recommended if data points are few), else will reduce number of knots for data size
i_want_increasing_II = 1 # if 1, reject II regimes for which injectivity is lower than the previous regime (recommended)
#
# Useful flags for injector_period_description column (so you can use words, not numbers, in the code body)
i_well_offline = 0    # used for PFOs
i_well_online = 1     # used for steady-state injection periods
i_well_transient = 2  # well performance is changing, unlabelled
i_well_beanup = 3     # well performance is changing, well is being beaned up from shut-in
i_well_beandown = 4   # well performance is changing, well is being beaned down to shut-in
i_well_beanops = 5    # well performance is changing, well is being beaned up/down from non-zero to non-zero rate
#
# Useful flags for P-Q stabilistaion type pick
i_stab_last = 0
i_stab_lastclip = 1
i_stab_minderv = 2
i_stab_mindervclip = 3
i_stab_longestdt = 4
#
# steady-state II flags (to describe PFO length prior to start-up)
i_sitype_poor = 0
i_sitype_good = 1
#
# QC scoring
i_min_clusters = 4          # number of clusters to be considered a "good" bean-up
f_min_cluster_power = 1     # qc score related to (1) (clusters/i_min_clusters)^f_min_cluster_power
f_CoV_penalty = 0.25        # qc score related to (2) sample coefficient of variation * f_CoV_penalty
#
# Well dictionary (this needs to be updated regularly)
#
# indexing for list that dictionary keys refer to
i_d_well_RMU = 0                         # RMU that the well belongs to
i_d_well_offset = 1                      # Producer(s) that best describes far-field / offset reservoir pressure (as a list, even for a single well)
i_d_well_gauge = 2                       # gauge used for well (use "DHG" or "WHG")
i_d_well_status_logic = 3                # method to pick well status (is 1, use rate = 0 as well as IMV/IWV for logic; this can help metered wells)
i_d_well_zthresh = 4                     # z-score threshold used to detect choke change events, recommend 2-3 (need higher value for wells with greater choke changes)
i_d_well_icv_delta_window = 5            # window to derive the delta ICV over, minutes (recomment 3-5)
i_d_well_icv_delta_smooth_window = 6     # rolling period to smooth the delta ICV over, minutes (recomment 3-5)
i_d_well_icv_threshold_smooth = 7        # window to smooth the binary choke change over for catching start-end of choke change periods, minutes (recommend 60-120)
i_d_well_i_shutin_dt = 8                 # shutin time to take injection back-pressure at for steady-state calculations, hours (recommend 48-120)
i_d_well_flow_period = 9                 # length of time (minutes) to consider a bean-up, suggest ~10 mins
i_d_well_freq_thresh = 10                # thresholding for rejecting spurious ICV integer thresholding to guess number of k-means clusters, recommend ~5-10
i_d_well_cluster_series = 11             # series to perform clustering on; 1 = ICV, 2 = ICV/pressure, 3 = ICV/rate, 4 = ICV/rate/pressure (recommended), 5 = rate/pressure
i_d_well_cluster_normalise = 12          # if 1, normalise the series data prior to the clustering; if 0 then use the raw series data for clustering
i_d_well_dbscan_PQ_eps = 13              # DBSCAN density clustering parameter for bean-up clustering (recommend ~0.1-0.2 for normalised data, ~2.5-7.5 for non-normalised data on rate/pressure; see PPT!)
i_d_well_dbscan_PQ_minsample = 14        # DBSCAN minimum number of samples for bean-up clustering (recommend 2-5)
i_d_well_beanup_preferred = 15           # Method picked for reporting back final bean-up P-Q clusters (1 = DBSCAN - recommended, 2 = k-means, 3 = integer ICV)
i_d_well_smallest_cluster = 16           # Smallest allowable P(Q) bean-up cluster size (recommend 5 - shorter clusters will be interpreted as noise)
i_d_well_monotonicity_method = 17        # Method to estimate monotonicity (0 = min (recommended), 1 = mean, 2 = max, 3 = median)
i_d_well_filter_behaviour = 18           # Method to filter poly-monotonic bean-ups (0 = take first zero rate point, 1 = take longest monotonic build (recommended))
i_d_well_stabilisation_method = 19       # Method used to pick stabilisation P-Q point (0 = last point, 1 = last point clipped to P10-90 - recommended, 2 = minimum 2D derivative dQ/dt.dP/dt, 
                                         #   3 = minimum 2D derivative dQ/dt.dP/dt clipped to P10-P90, 4 = longest common dt point)
i_d_well_dbscan_II_eps = 20              # injectivity index distance to consider P(Q) point to be in the same injection regime, recommend 10-20 (stb/d/psi)
i_d_well_dbscan_II_minsamples = 21       # min samples for II clustering (i.e. must have at least N stabilisation points to consider a flow regime; recommend 1 for wells with few ICV changes otherwise >2)
i_d_well_dbscan_discretisation = 22      # method to generate point II for clustering; 0 = first-order backwards (recommended), 1 = first-order backwards to zero point, 2 = instantanous P/Q (not recommended)
i_d_well_minfracpressure = 23            # minimum pressure (barg) to be considered a viable frac pressure to avoid spurious temperature dependent II
i_d_well_maxII = 24                      # maximum injectivity value (m3/hr/bar) to be considered a viable II (for reference, 100 stb/d/psi = 9.6 m3/hr/bar)
i_d_well_maxQfrac = 25                   # maximum frac breakover rate (m3/hr/bar) to be considered a viable II (for reference, 200 m3/hr = 30 mbd)
i_d_well_maxPfrac = 26                   # maximum frac breakover pressure (barg) to be considered viable
#
# dictionary data (example)
d_well_data = {
    #      |------0-1-------|--2--|------3-8------|----------9-19------------|-------------20-25-------------|
    #      |------RMU-------|Gauge|<--Status,PFO->|<--Bean-up clustering---->|--------II cluster/clip--------|
    'I01' : ['S1'  ,['P01'] ,'WHG',1,2.5,3,5,90,48,10,10,4,1,0.15,2,1,5,0,1,1,15.0,1,0,100.0,10.0,200.0,275.0],
    #                                                    N  EPS m          II m
}
#
# value-to-number dictionary (to fix the Palantir "feature" introduced on enumerated data series)
d_valve_enumerate = {
    "CLOSED":[66364536],    # also takes 0, don't see this on GL valves
    "OPEN":[-1010875264],   # also takes 1, don't see this on GL valves
    "Closed":[66364536],    # also takes 0, don't see this on GL valves
    "Open":[-1010875264],   # also takes 1, don't see this on GL valves
    "UNDEFINED":[255],
    "Undefined":[-334512128]
    }
#
# global function for interpolating across time series (using scipy.interpolate)
#
# takes inputs: (1) target x <target df time> (2) defined x <original df time> (3) defined y <original df value>
#               (4) interp type ('linear','previous','nearest'; fallback is linear interpolation)
# e.g. interpolate_df_col(df['time'],other_df['time'],other_df['col'],'linear')
def interpolate_df_col(target_time,def_time,def_y,interp_type):
    #
    # check interpolation type and assign linear if incorrect value assigned
    if (interp_type != 'previous') and (interp_type != 'nearest'):
        interp_type = 'linear'
    #
    # get reference datetime as minimum of two time sets
    t_ref = min(min(target_time),min(def_time))
    #
    # convert target times to floats (days from ref time) so scipy can interpolate
    f_def_time = (def_time - t_ref).dt.total_seconds() / 86400.0
    f_target_time = (target_time - t_ref).dt.total_seconds() / 86400.0
    #
    # set up scipy interpolation function (note if we try to interpolate out of bounds, this will set to 0.0)
    f = interpolate.interp1d(f_def_time,def_y,kind=interp_type,bounds_error=False,fill_value=0.0)
    #
    # return the new interpolated array
    return f(f_target_time)
#

In [ ]:
def process_well_data_DHG(Pull_well_data_DHG, list_of_wells):
    #
    # Script to loop through the well data and do some simple pre-processing
    #    e.g. clip outliers, select the main pressure/temperature tag, derive well status etc.
    #
    #    -> runs on the DHG wells, to remove memory limits on saving dfs <-
    #
    # Also labels all periods as bean-up, steady-state injection, PFO, bean-down etc.
    #    (these correlate to the global code integers i_well_offline, i_well_beanup etc.)
    #
    df = Pull_well_data_DHG.copy(deep=True)
    #
    # build list of wells - for now, do this with the available injection wells as we don't have a "well" column
    # (each name is embedded within the column name)
    my_wells = list_of_wells.columns.tolist()
    #
    for i_df,well in enumerate(my_wells): # we defined the list my_wells above
        #
        # get the list of column names
        col_well_name = well
        col_well_IMV = str(well+'_Master_valve')
        col_well_IWV = str(well+'_Wing_valve')
        col_well_ICV = str(well+'_Choke_valve')
        col_well_q = str(well+'_Inj_water_rate')
        if d_well_data[well][i_d_well_gauge] == 'DHG':
            col_well_p = str(well+'_BHP')
            col_well_T = str(well+'_BHT')
        else:
            col_well_p = str(well+'_WHP')
            col_well_T = str(well+'_WHT')
        #
        # wrap all the building around a try-except clause, so any wells that we didn't pull data for but are still in the 
        # dictionary (used to build the my_wells list) won't break the code
        try:
            # build well df
            df_well = df[['datetime',col_well_IMV,col_well_IWV,col_well_ICV,col_well_q,col_well_p,col_well_T]].copy(deep=True)
            #
            # label columns
            df_well.columns = ['datetime','IMV','IWV','ICV','rate','pressure','temperature']
            #
            # insert column for well name
            df_well.loc[:,'well'] = [well]*len(df_well)
            #
            ######################################################################
            # 1. Prepare the input data columns (clip outliers, add well status) #
            ######################################################################
            #
            # clip outliers
            df_well.loc[:,'rate'] = df_well['rate'].clip(upper=300.0,lower=0.0).replace(300.0,np.nan)                        # note 0 could be a rate here, so don't replace with NaN
            df_well.loc[:,'pressure'] = df_well['pressure'].clip(upper=375.0,lower=30.0).replace([375.0,30.0],np.nan)
            df_well.loc[:,'temperature'] = df_well['temperature'].clip(upper=70.0,lower=0.0).replace([70.0,0.0],np.nan)
            df_well.loc[:,'ICV'] = df_well['ICV'].clip(upper=100.0,lower=0.0).replace([100.0,0.0],np.nan)
            #
            # drop any row that contains a NaN (should drop clip data)
            df_well.dropna(inplace=True)
            #
            # go through the data and determine well status
            # set ICV to -10 if rate is zero as we'll use this to detect bean-ups where only the IWV is opened
            well_status = []
            icv_status = []
            #
            # loop over well df and build two lists of (1) well status (2) ICV at current time (overwritten with -10 if well is offline)
            #
            # option 1 - just use IWV/IMV to derive well status
            if d_well_data[well][i_d_well_status_logic] == 0: 
                #
                df_well.loc[:,'status'] = [i_well_online if (x<0 and y<0) else i_well_offline for x,y in zip(df_well['IMV'],df_well['IWV'])]
                #
            # option 2 - also use rate to derive status (useful for metered wells as additional check)
            else:
                #
                df_well.loc[:,'status'] = [i_well_online if (x<0 and y<0 and z>1.0) else i_well_offline for x,y,z in zip(df_well['IMV'],df_well['IWV'],df_well['rate'])]
                #
            #
            # overwrite ICV if status is zero
            df_well.loc[:,'ICV'] = [-10.0 if (stat == i_well_offline) else icv for stat,icv in zip(df_well['status'],df_well['ICV'])]
            #
            # drop the IMV/IWV data, don't need this any more
            df_well.drop(columns=['IMV','IWV'],inplace=True)
            #
            # sort df by datetime, just to make sure
            df_well.sort_values(by='datetime',inplace=True)
            #
            #########################################################################
            # 2. Generate change-of-ICV binary column (to detect periods of change) #
            #########################################################################
            #
            # generate delta ICV column (which is subsequently smoothed - well dictionary contains window to delta/smooth over)
            df_well.loc[:,'icv_delta'] = df_well['ICV'].diff(periods=d_well_data[well][i_d_well_icv_delta_window]).rolling(window=d_well_data[well][i_d_well_icv_delta_smooth_window],center=True).mean()
            #
            # generate z scores of change of ICV (should be strongly centred around ~0 with some noise)
            df_well.loc[:,'zscores_delta'] = np.abs(stats.zscore(df_well['icv_delta'].fillna(0)))
            #
            # threshold on z-scores, i.e. look for outliers (may need to adjust zthresh if well is particularly active, again z clip is stored in well dictionary)
            # -> also make sure well is online so we don't detect choke changes from e.g. integrity tests whilst the well is offline
            df_well.loc[:,'icv_delta_threshold'] = np.where((df_well.zscores_delta > d_well_data[well][i_d_well_zthresh])&(df_well.status == 1),1,0)
            #
            # generate a smoothed pick up of the choke change detection (so that we can catch just before a bean-up, for example)
            df_well.loc[:,'icv_delta_threshold_smoothed'] = df_well['icv_delta_threshold'].rolling(window=d_well_data[well][i_d_well_icv_threshold_smooth],center=True).mean().fillna(0.0)
            #
            # finally, create binary map of choke change periods as the smoothing will smear out the 0>1 piecewise function generated in df_well['icv_delta_threshold'] column
            df_well.loc[:,'icv_delta_threshold_binary'] = [1 if (icv_thresh > 0.0) else 0 for icv_thresh in df_well['icv_delta_threshold_smoothed']]
            #
            # drop extra columns now that we have the binary flag (comment these out if you want to save column for debugging)
            df_well.drop('zscores_delta',axis=1,inplace=True)
            df_well.drop('icv_delta',axis=1,inplace=True)
            df_well.drop('icv_delta_threshold',axis=1,inplace=True)
            df_well.drop('icv_delta_threshold_smoothed',axis=1,inplace=True)

            # drop any rows with NaNs from the dataframe
            df_well.dropna(inplace=True)
            #
            #####################################################################
            # 3. Label each period as online, offline, bean-up, bean-down etc.  #
            #####################################################################
            #
            # we will first loop over the threasholded map and increment when we find a new period, whether on or off
            # -> this will create a unique list [0,1,2,...,n] where of all periods
            # -> we'll label the transient periods in more detail afterwards
            #
            # initialise loop lists
            my_period_ID_list = [0]*len(df_well)           # list of unique period IDs (0,1,2,...,n)
            my_period_description_list = [0]*len(df_well)  # will contain integer flag for period type (st-state, bean-up, bean-down, PFO etc.)
            my_thresholds = df_well['icv_delta_threshold_binary'].tolist()
            my_well_status = df_well['status'].tolist()
            #
            # initialise  loop / list counter
            current_period = 1
            prev_thresh = my_thresholds[0]
            #
            # loop over all periods and label as steady- or unsteady-state
            for i,thresh in enumerate(my_thresholds):
                #
                # (1) injector is in a transient state (bean-up, bean-down, changing operational ICV setting)
                if thresh == 1:
                    #
                    if (prev_thresh == 0) & (i > 0): # if now transient and it wasn't before, increment steady-state counter (ignore first data point in list)
                        current_period += 1
                    #
                    my_period_ID_list[i] = current_period
                    my_period_description_list[i] = i_well_transient
                    #
                #    
                # (2) injector is in a steady-state period (online or offline)    
                else:
                    #
                    if (prev_thresh == 1) & (i > 0): # if now steady and the last period was transient, increment the period counter (ignore first data point in list)
                        current_period += 1
                    #
                    my_period_ID_list[i] = current_period
                    #
                    if my_well_status[i] == 0:
                        my_period_description_list[i] = i_well_offline
                    else:
                        my_period_description_list[i] = i_well_online
                #
                # update previous thresholded value
                prev_thresh = thresh
                #
            #
            # add the period lists to the data frame
            df_well.loc[:,'injector_period_ID'] = my_period_ID_list
            df_well.loc[:,'injector_period_description'] = my_period_description_list
            #
            # check all elements have a period ID
            if len(df_well[df_well['injector_period_ID']==0]) > 0:
                print(str('Warning: '+str(len(df_well[df_well['injector_period_ID']==0]))+' time periods unidentified in well '+well))
            #
            # set up blank dictionary that we'll map period number to period type on
            d_period_update = {}
            #
            # loop over the transient periods and map transient period to type of period
            for inj_period in df_well[df_well['injector_period_description']==i_well_transient]['injector_period_ID'].unique():
                #
                # get the sub-df for this injection period
                temp_df = df_well[df_well['injector_period_ID']==inj_period]
                #
                # distinguish between bean-up/down and operational choke change by zero rate length (i.e. in the ops one, we're going from finite-to-finite rate)
                if len(temp_df[temp_df['rate']<0.1]) > 0:
                    #
                    # if first rate is zero, well is being beaned up
                    # -> this logic will fail if the ICV threshold smoothing window is too short to catch zero rate at start-up
                    if temp_df['rate'].iloc[0] < 0.1:
                        d_period_update[inj_period] = i_well_beanup
                    else:
                        d_period_update[inj_period] = i_well_beandown
                else:
                    # no zero rate points > injector ICV is being changed during normal operations
                    d_period_update[inj_period] = i_well_beanops
            #        
            # go through the periods and replace the injector period description
            for inj_period in d_period_update:
                df_well.loc[df_well['injector_period_ID']==inj_period,'injector_period_description'] = d_period_update[inj_period]
            #
            # warn if we have remaining transient periods unlabelled
            if len(df_well[df_well['injector_period_description']==i_well_transient])>0:
                print(str('Warning: '+str(len(df_well[df_well['injector_period_description']==i_well_transient]))+' transient injection periods are unlabelled in well '+well))
            #
            # drop the threshold detection column
            df_well.drop('icv_delta_threshold_binary',axis=1,inplace=True)
            #
            ########################################
            # 4. Output some useful details to log #
            ########################################
            #
            # print some statistics to the screen
            print(str('Data summary for well: '+well))
            print(str('  Choke change periods detected: '+str(len(df_well['injector_period_ID'].unique()))))
            print(str('    ... of which are bean-ups: '+str(len(df_well[df_well['injector_period_description']==i_well_beanup]['injector_period_ID'].unique()))))
            print(str('    ... of which are bean-downs: '+str(len(df_well[df_well['injector_period_description']==i_well_beandown]['injector_period_ID'].unique()))))
            print(str('    ... of which are shut-in periods: '+str(len(df_well[df_well['injector_period_description']==i_well_offline]['injector_period_ID'].unique()))))
            print(str('    ... of which are online periods: '+str(len(df_well[df_well['injector_period_description']==i_well_online]['injector_period_ID'].unique()))))
            print(str('    ... of which are ops change periods: '+str(len(df_well[df_well['injector_period_description']==i_well_beanops]['injector_period_ID'].unique()))))
            print(str('    ... of which are unknown transient periods: '+str(len(df_well[df_well['injector_period_description']==i_well_transient]['injector_period_ID'].unique()))))
            print('')
            #
            # output to main df
            try :
                df_output = pd.concat([df_output,df_well],ignore_index=True)
            except:
                df_output = df_well.copy(deep=True)
        #
        # case where we have a well in dictionary but data wasn't pulled, pass this well
        except:
            pass
        #
    #
    # # replace empty values with NaNs to avoid Apache Parquet dying a premature death
    # df_global_ststate_II.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    #
    return df_output

In [ ]:
def st_st_II_tracker_DHG(process_well_data_DHG):
    #
    # Transform to process the steady-state periods and derive injectivity each steady-state point (DHG wells only for memory limits!)
    #
    # (will take every 10th point to reduce the data base size)
    #
    # merge together the DHG/WHG input data frames
    df = process_well_data_DHG.copy(deep=True)
    #
    for i_df,well in enumerate(df['well'].unique()):
        #
        # take sub-slice df of this well
        df_well = df[df['well']==well].copy(deep=True)
        #
        # get list of steady-state and shut-in periods
        l_ststate = df_well[df_well['injector_period_description']==i_well_online]['injector_period_ID'].unique().tolist()
        l_shutins = df_well[df_well['injector_period_description']==i_well_offline]['injector_period_ID'].unique().tolist()
        #
        # reset counter for number of processed steady-state periods
        i_count_df = 0
        #
        # loop over the steady-state periods and derive II from each one
        for ststate_period in l_ststate:
            #
            # only bother if the period is >1 into the flow (as we need at least two states - shut-in and bean-up - prior to a st-state period)
            if ststate_period > 2:
                #
                # get the previous periods - must follow a bean-up and a shut-in to be considered a "good" steady-state pick
                previous_period_1 = df_well[df_well['injector_period_ID']==ststate_period-1]['injector_period_description'].iloc[0]
                previous_period_2 = df_well[df_well['injector_period_ID']==ststate_period-2]['injector_period_description'].iloc[0]
                #
                # if previous two periods are bean-up and offline, analyse the II for this steady-state period
                if previous_period_1 == i_well_beanup and previous_period_2 == i_well_offline:
                    #
                    # glue together the periods and pick zero rate times (as we may have some zero rates during the bean-up period for example)
                    df_shutin_period = df_well[df_well['injector_period_ID'].isin([ststate_period-3,ststate_period-2,ststate_period-1])].sort_values(by='datetime')
                    df_shutin_period = df_shutin_period[df_shutin_period['status']==i_well_offline]
                    #
                    # grab the start time and look for an entry n-hours in front
                    pfo_start_time = df_shutin_period['datetime'].iloc[0]
                    pfo_search_time = pfo_start_time + pd.Timedelta(d_well_data[well][i_d_well_i_shutin_dt],'hours')
                    #
                    # check if the search time is in the shut-in df by comparing to the search datetime to the last datetime in the df
                    if df_shutin_period['datetime'].iloc[-1] < pfo_search_time:
                        #
                        # if target shut in time not in list (PFO too short), just take the last value as the reservoir pressure
                        my_sip = df_shutin_period['pressure'].iloc[-1]   # pressure taken
                        my_sit = i_sitype_poor                           # label as poor (inconsistent) pressure
                        my_sid = df_shutin_period['datetime'].iloc[-1]   # datetime used for reservoir pressure estimate 
                    #
                    else:
                        #
                        # if target shut in time is in the list (PFO same/longer than required), take the target PFO shut-in time
                        try:
                            my_sip = df_shutin_period['pressure'].loc[df_shutin_period['datetime']==pfo_search_time].iloc[0]
                            my_sid = pfo_search_time
                            my_sit = i_sitype_good
                        except: # may fail for some reason, e.g. inconvenient data outage
                            my_sip = df_shutin_period['pressure'].iloc[-1]
                            my_sid = df_shutin_period['datetime'].iloc[-1]
                            my_sit = i_sitype_poor
                    #
                    # remember the length of the shut in
                    my_si_length_d = (max(df_shutin_period['datetime']) - min(df_shutin_period['datetime'])).total_seconds() / 86400.0
                    #
                    # now, pull the steady-state data
                    df_ststate = df_well[df_well['injector_period_ID']==ststate_period]
                    #
                    # empty lists to store the generated II
                    my_II_metric_list = []
                    my_II_field_list = []
                    my_date_list = []
                    #
                    # loop over the steady-state data and generate the II with the shut-in pressure pulled above
                    for i,row in df_ststate.iterrows():
                        #
                        # only pull the data on certain samples (i_sample_SS)
                        if i % i_sample_SS == 0:
                            II_metric = row['rate'] / (row['pressure'] - my_sip)
                            my_II_metric_list.append(II_metric)
                            my_II_field_list.append(II_metric*f_II_metric_to_field)
                            my_date_list.append(row['datetime'])
                    #
                    # slice df_ststate on the time points picked by the modulo logic
                    df_ststate = df_ststate[df_ststate['datetime'].isin(my_date_list)]
                    #
                    # add II to new dataframe
                    df_ststate.loc[:,'II_metric'] = my_II_metric_list
                    df_ststate.loc[:,'II_field'] = my_II_field_list
                    df_ststate.loc[:,'PRES_shutin'] = [my_sip]*len(df_ststate)
                    df_ststate.loc[:,'PRES_type'] = [my_sit]*len(df_ststate)
                    df_ststate.loc[:,'PRES_date'] = [my_sid]*len(df_ststate)
                    df_ststate.loc[:,'shutin_length_d'] = [my_si_length_d]*len(df_ststate)
                    #print(well, ststate_period, my_sid)
                    #
                    # finally, write st-state df to overall one for this well
                    try:
                        df_ststate_II = pd.concat([df_ststate_II,df_ststate])
                    except:
                        df_ststate_II = df_ststate.copy(deep=True)
                    #
                    # advance the steady-state counter
                    i_count_df += 1
                    #
                # end check for previous period character
            # end that st-state period > 2
        # end loop over st-state periods for this well
        #
        # roll up the well st-state II dfs into one global df
        try:
            df_global_ststate_II = pd.concat([df_global_ststate_II,df_ststate_II],ignore_index=True)
        except:
            df_global_ststate_II = df_ststate_II.copy(deep=True)
        #
        # print output message to log
        print(str('Processed steady-state injectivity history for well: '+well))
        print(str('Total number of II periods: '+str(len(l_ststate))))
        print(str())
        #
    # end loop over wells
    #
    # replace empty values with NaNs to avoid Apache Parquet dying a premature death
    df_global_ststate_II.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    #
    return df_global_ststate_II

In [ ]:
def pick_clusters_DHG(process_well_data_DHG):
    #
    # Code transform to take the processed bean-up data and determine cluster points for each P(Q) cluster,
    #   using the DBSCAN clustering method (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)
    #
    # This is pretty lengthy as there are several cleaning stages applied. Read the PPT "manual" if in doubt ...
    #
    # derive the bean-up data set from the processed df
    df = process_well_data_DHG.copy(deep=True).sort_values(by='datetime')
    #
    for i_df,well in enumerate(df['well'].unique()):
        #
        # take sub-slice df of this well for the bean-ups only
        df_well = df[(df['well']==well) & (df['injector_period_description']==i_well_beanup)].copy(deep=True)
        #
        # get list of steady-state and shut-in periods
        l_beanup_periods = df_well['injector_period_ID'].unique().tolist()
        #
        # remember the number of bean-up periods
        i_no_beanups = len(l_beanup_periods)
        #
        #############################################################
        # 1. Do some early data cleaning on the identified bean-ups #
        #    - eliminate periods with too short a flowing time      #
        #    - pick longest continuous period if bean-up was        #
        #      interrupted                                          #
        #############################################################
        #
        # now, we eliminate any periods with flowing time < n minutes for noisy data
        i_flow_period = d_well_data[well][i_d_well_flow_period]
        for beanup in l_beanup_periods:
            if len(df_well[(df_well['injector_period_ID']==beanup)&(df_well['rate']>0.0)]) < i_flow_period:
                l_beanup_periods.remove(beanup)
        #
        # loop through the long-enough beanups and detect the longest flowing period that defines the main (online) bean-up
        for i_df_beanups, beanup in enumerate(l_beanup_periods):
            #
            # slice the main df on this beanup; reset the index so we can slice on this later
            df_beanup = df_well[df_well['injector_period_ID']==beanup].reset_index(drop=True)
            #
            # initialise beanup loop constants
            i_start_from_shutin = 1 # 1 if starts from shut-in state
            i_status_changes = 0    # well status will be on or off, that's what we're tracking here for interrupted bean-ups
            i_prev_status = df_beanup['status'].iloc[0]  # should equal 1 for the first (1/0 for on/off)
            i_period_length = 0
            #
            # dictionary of sub-periods that we'll build
            d_periods = {}
            #
            # integer slicing for list that dictionary key will refer to
            i_d_periods_status = 0
            i_d_periods_length = 1
            i_d_periods_dfindex = 2
            #
            # now loop over the rows for this one beanup and build a dictioary of period:[status, length at that status, df index]
            for i_dx, row in df_beanup.iterrows():
                #
                # bean-up should start from shut-in - if not then flag
                if (i_dx == 0) and (i_prev_status == 1):
                    print("Warning - beanup doesn't appear to start from shut-in")
                    print(str("Well = "+well+"; beanup ID = "+str(beanup)))
                    i_start_from_shutin = 0
                #
                # count number of periods and length of period
                if row['status'] != i_prev_status:
                    d_periods[i_status_changes] = [i_prev_status,i_period_length,i_dx] # store in dictionary [previous stat, length at previous stat, row number]
                    i_period_length = 0                 # reset period length
                    i_status_changes += 1               # increment status change counter
                    i_prev_status = row['status']       # update previous status to this row
                else:
                    i_period_length += 1                # if status is stlil the same, just increment the period length
                #
            #
            # add the last loop entry to the dictionary too
            d_periods[i_status_changes] = [i_prev_status,i_period_length,i_dx]
            #
            # if there are more than two status changes observed in the bean-up (offline > online bean-up) then
            #   just take the longest online length and the preceding shut-in
            #
            # -> we'll do this in a loop, so initialise some loop variables
            my_longest_period = 0
            len_longest_period = 0
            #
            # here is the loop - there's probably a more efficient way to do this!
            if len(d_periods) > 2:
                #
                # loop variables
                my_longest_period = 0
                len_longest_period = 0
                #
                # loop over each period key in dictionary and find the longest online period
                for i_period,period in enumerate(d_periods):
                    #
                    # if well doesn't start from shut-in, ignore the first flowing period
                    if (i_start_from_shutin == 0) and (i_period == 0):
                        continue # move on to the next iteration
                    else:
                        if (d_periods[period][i_d_periods_status] == 1) and (d_periods[period][i_d_periods_length] > len_longest_period):
                            my_longest_period = period                                   # update reference to longest period
                            len_longest_period = d_periods[period][i_d_periods_length]   # update length of longest period
                #
                if len_longest_period == 0: # failed to find anything online, whoops
                    #
                    print('Warning - multiple subperiods present, but failed to find longest')
                    print(str("Well = "+well+"; beanup ID = "+str(beanup)))
                    print('Dictionary of subperiods:')
                    print(d_periods)
                    df_index_from = 0
                    df_index_to = 0
                else:
                    #
                    # otherwise, set df index from longest period and previous one
                    #
                    # first index for slice is the end of the shut-in index minus the length of that period (to pick start of shut-in)
                    #   (need to subtract one because Python slicing is [from-including,to-not-including])
                    df_index_from = max(0,d_periods[my_longest_period-1][i_d_periods_dfindex]-d_periods[my_longest_period-1][i_d_periods_length]-1)
                    # 
                    # second index for slice is end point of flowing index
                    df_index_to = min(len(df_beanup),d_periods[my_longest_period][i_d_periods_dfindex])
                    #
            else:
                # 2 periods - use all the df
                if i_start_from_shutin == 1:
                    df_index_from = 0
                    df_index_to = len(df_beanup)
                # if 2 periods but the first is flowing, slice on nothing (empty df > will be cleansed and ignored)
                else:
                    df_index_from = 0
                    df_index_to = 0
            #
            # now slice the beanup data-frame on the longest online period
            df_beanup = df_beanup.iloc[df_index_from:df_index_to]
            #
            # save beanup data
            if i_df_beanups == 0:
                df_beanup_filtered = df_beanup.copy(deep=True)
            else:
                try:
                    df_beanup_filtered = pd.concat([df_beanup_filtered,df_beanup])
                except:
                    df_beanup_filtered = df_beanup.copy(deep=True)
            #
        # end loop over beanup to determine longest online period
        #
        ###########################################################
        # 2. Perform clustering on the data; offer ICV threshold, #
        #       k-means clustering and DBSCAN (preferred)         #
        ###########################################################
        #
        # first, build list of integer-rounded ICVs
        df_beanup_filtered.loc[:,'icv_int'] = df_beanup_filtered['ICV'].astype('int64').tolist()
        #
        # update the beanup list post-filtering and clean up
        l_beanup_periods = df_beanup_filtered['injector_period_ID'].unique()
        #
        # loop over each beanup again and cluster
        for i_df_beanups,beanup in enumerate(l_beanup_periods):
            #
            # take df of the beanup
            df_beanup = df_beanup_filtered[df_beanup_filtered['injector_period_ID'] == beanup].reset_index(drop=True)
            #
            # reset list for frequency of unique thresholded ICVs
            l_ICV_freq = []
            #
            # determine list of unique integer ICVs and number of df entries for each one (i.e. how many minutes at each position)
            for icv in df_beanup['icv_int'].unique():
                l_ICV_freq.append([icv,len(df_beanup[df_beanup['icv_int']==icv])])
            #
            # append list to df of two columns - ICV and number of occurrences- and sort by occurrences
            df_ICV_freq = pd.DataFrame(np.array(l_ICV_freq),columns=['icv','thresh']).sort_values(by='thresh')
            #
            # filter df by threshold of number of occurrences (i.e. length of time at this position)
            df_ICV_freq = df_ICV_freq[df_ICV_freq['thresh'] > d_well_data[well][i_d_well_freq_thresh]]
            #
            # use the length of this thresholded df to estimate number of clusters for other methods (ICV / k-means) downstream
            n_clusters = len(df_ICV_freq)
            #
            # loop dictionary / list initialisation
            d_ICV_cluster = {}
            my_cluster_number = 0
            ICV_cluster_list = []
            #
            # will build list of thresholded ICV cluster values and assign unique cluster values on this for the beanup (first clustering method)
            for icv in df_beanup['icv_int'].tolist():
                if icv in d_ICV_cluster: 
                    # assign existing value to cluster if it's already in the dictionary of unique ICVs
                    ICV_cluster_list.append(d_ICV_cluster[icv])      # add the unique cluster number to list previously assigned to this integer ICV
                else:
                    # if ICV is in the thresholded list, keep the ICV and store value in dictionary
                    if icv in df_ICV_freq['icv'].tolist():
                        d_ICV_cluster[icv] = my_cluster_number       # assign unique cluster number to map ICV > cluster
                        my_cluster_number += 1                       # advance unique cluster number
                        ICV_cluster_list.append(d_ICV_cluster[icv])  # add the unique cluster number to list
                    #
                    # if not in thresholded list, assign a -1 value to label as noise
                    else: 
                        d_ICV_cluster[icv] = -1
                        ICV_cluster_list.append(d_ICV_cluster[icv])
            #
            # store ICV threshold clusters in df
            df_beanup['icv_int_clustering'] = ICV_cluster_list
            #
            # build the input data sets for k-means / DBSCAN clustering methods (both need same style of input)
            #
            # get the main series data for the clusters
            if d_well_data[well][i_d_well_cluster_series] == 1: # ICV only
                cluster_data = np.array(df_beanup['ICV'])
                cluster_data = np.reshape(cluster_data,(len(cluster_data),1)) # need to reshape 1D array
            elif d_well_data[well][i_d_well_cluster_series] == 2: # ICV / pressure
                cluster_data = np.array(df_beanup[['ICV','pressure']])
            elif d_well_data[well][i_d_well_cluster_series] == 3: # ICV / rate
                cluster_data = np.array(df_beanup[['ICV','rate']])
            elif d_well_data[well][i_d_well_cluster_series] == 5: # rate / pressure
                cluster_data = np.array(df_beanup[['rate','pressure']])
            else:                                                 # (4 - default option) ICV/rate/pressure
                cluster_data = np.array(df_beanup[['ICV','rate','pressure']])
            #
            # normalise the data if requested
            if d_well_data[well][i_d_well_cluster_normalise] == 1:
                cluster_data = StandardScaler().fit_transform(cluster_data)
            #
            # perform k-means clustering on the data
            try:
                kmeans = KMeans(n_clusters=n_clusters).fit(cluster_data)
                df_beanup.loc[:,'km_cluster_labels'] = kmeans.labels_
            except:
                df_beanup.loc[:,'km_cluster_labels'] = [-1]*len(df_beanup)
            #
            # perform DBSCAN clustering on the data
            try:
                dbscan_cluster = DBSCAN(eps = d_well_data[well][i_d_well_dbscan_PQ_eps], min_samples = d_well_data[well][i_d_well_dbscan_PQ_minsample]).fit(cluster_data)
                df_beanup.loc[:,'dbscan_labels'] = dbscan_cluster.labels_
            except:
                df_beanup.loc[:,'dbscan_labels'] = [-1]*len(df_beanup)            
            #
            # store the beanup df
            if i_df_beanups == 0:
                df_clustered = df_beanup.copy(deep=True)
            else:
                try:
                    df_clustered = pd.concat([df_clustered,df_beanup])
                except:
                    df_clustered = df_beanup.copy(deep=True)
            #
        # end of loop over beanups to perform clustering
        #
        ##############################
        # 3. Clean up clustered data #
        ##############################
        #
        # set up some loop integers / lists
        i_branch_cluster = 0   # use this to refer to list of sub-clusters (dictionary that will be built in the following loop)
        i_branch_index = 1     # use this to refer to list of sub-clusters (dictionary that will be built in the following loop)
        l_new_clusters = []    # use this to build the new list of cluster values that will be added to df_clustered
        #
        # set up the preferred cluster column
        if d_well_data[well][i_d_well_beanup_preferred] == 1:   # use DBSCAN clusters
            df_clustered.loc[:,'cluster'] = df_clustered['dbscan_labels'].tolist()
        elif d_well_data[well][i_d_well_beanup_preferred] == 2: # use k-means clusters
            df_clustered.loc[:,'cluster'] = df_clustered['km_cluster_labels'].tolist()
        else:                                                   # use ICV integer labels
            df_clustered.loc[:,'cluster'] = df_clustered['icv_int_clustering'].tolist()
        #
        # loop over the beanups and clean up clustering - sub-divide those that occur in more than one discrete period
        for beanup in df_clustered['injector_period_ID'].unique():
            #
            # get the beanup data
            df_cluster_beanup = df_clustered[df_clustered['injector_period_ID']==beanup].copy(deep=True)
            #
            # get the unique cluster values for this beanup
            l_beanup_cluster = df_cluster_beanup['cluster'].unique()
            #
            # initialise subdivide cluster number to max of list + 1
            i_cluster_subdivide = max(l_beanup_cluster)+1
            #
            # set up dictionaries for loop within this beanup
            d_found_clusters = {}     # dictionary of cluster:row index
            d_branched_clusters = {}  # dictionary of cluster:[[sub-cluster-1,sub-index-1],...,[sub-cluster-n,sub-index-n]]
            #
            # now loop over the df (sort by datetime just to make sure ...) and sub-divide if a second discrete period is found
            for i,row in df_cluster_beanup.iterrows():
                #
                row_cluster = row['cluster']
                #
                # logic branch if cluster has been found already (ignoring noise points)
                if (row_cluster in d_found_clusters) and (row_cluster >= 0):
                    #
                    # if cluster has been found, check the last cluster element was the entry before
                    if d_found_clusters[row_cluster] == i-1:
                        #
                        d_found_clusters[row_cluster] = i     # if so, update dictionary to reflect this as last element
                        l_new_clusters.append(row_cluster)    # store current cluster as correct
                        #
                    #
                    # if it wasn't the last entry, needs to be a new hierarchical cluster
                    else:
                        # firstly, check if the cluster has already been sub-divided
                        if row_cluster in d_branched_clusters:
                            #
                            # if cluster is sub-branched already, see if one of the sub-branches has the index-1 entry
                            i_sub_cluster_identified = 0
                            #
                            # subclusters are stored as d[row_cluster] = [[sub_cl_1,sub_index_1],[sub_cl_2,sub_index_2],...]
                            # dictionary of a list of lists ... my head's exploding
                            for i_sub_cluster,l_sub_cluster in enumerate(d_branched_clusters[row_cluster]):
                                #
                                sub_cluster_value = l_sub_cluster[i_branch_cluster]
                                sub_cluster_index = l_sub_cluster[i_branch_index]
                                #
                                # if the last index was used anywhere, don't need to add a new sub-cluster
                                if sub_cluster_index == i-1:
                                    #
                                    # update index in dictionary
                                    d_branched_clusters[row_cluster][i_sub_cluster][i_branch_index] = i
                                    # add sub cluster index to list
                                    l_new_clusters.append(sub_cluster_value)
                                    # update flag to state sub cluster was found
                                    i_sub_cluster_identified = 1
                                    # don't need to loop any more if we have found the sub-cluster (and run risk of duplicates)
                                    break
                            #
                            # if cluster is not found in existing sub-clusters, write the new sub cluster to the dictionary
                            if i_sub_cluster_identified == 0:
                                #
                                # generate the new sub-cluster label
                                i_cluster_subdivide += 1
                                # add the cluster to the branch dictionary
                                d_branched_clusters[row_cluster] = []
                                # append the sub-cluster name and index to the branch dictionary entry
                                d_branched_clusters[row_cluster].append([i_cluster_subdivide,i])
                                # add the new cluster to list
                                l_new_clusters.append(i_cluster_subdivide)
                                #
                            #
                        #
                        # if cluster hasn't been sub-divided before, split it here
                        else:
                            #
                            # generate the new sub-cluster label
                            i_cluster_subdivide += 1
                            # add the cluster to the branch dictionary
                            d_branched_clusters[row_cluster] = []
                            # append the sub-cluster name and index to the branch dictionary entry
                            d_branched_clusters[row_cluster].append([i_cluster_subdivide,i])
                            # add the new cluster to list
                            l_new_clusters.append(i_cluster_subdivide)
                            #
                        # if row_cluster in d_branched_clusters
                    # if d_found_clusters[row_cluster] == i-1:
                #
                # logic branch for cluster that hasn't been found already (or is noise)
                else:
                    if row_cluster < 0:
                        # if noise, don't bother to add
                        l_new_clusters.append(row_cluster) # store current cluster as correct
                    else:
                        # if not noise, add cluster to the dictionary and set index to dictionary value
                        d_found_clusters[row_cluster] = i
                        l_new_clusters.append(row_cluster) # store current cluster as correct
                    #
                #
            # end of loop over bean-up to build list of subclusters
        # end of loop over beanups to clean up clusters
        #
        # append the list of new clusters to the df
        df_clustered.loc[:,'new_clusters'] = l_new_clusters
        #
        # finally, go through the bean-ups and overwrite any small clusters with the noise (-1) flag
        i_min_cluster_size = d_well_data[well][i_d_well_smallest_cluster]  # minimum size for cluster
        l_filtered_clusters = []                                           # initialise list of filtered cluster values
        #
        for beanup in df_clustered['injector_period_ID'].unique():
            #
            # get sliced df for this beanup
            df_beanup = df_clustered[df_clustered['injector_period_ID']==beanup]
            #
            # get the list of clusters
            l_clusters = df_beanup['new_clusters'].unique().tolist()
            #
            # copy the list, Python doesn't like removing elements from iterated lists (fair enough)
            #l_clusters_copy = l_clusters.copy() # should work on Python 3.3 onwards, odd
            l_clusters_copy = list(l_clusters)   # old Python syntax that works
            #
            # loop over clusters and remove clusters that are too small
            for cluster in l_clusters:
                #
                # don't remove noise (i.e. only use clusters with zero/positive value)
                # also don't remove data with zero rate, regardless of length, as we'll need this for initial pressure guess
                if (len(df_beanup[df_beanup['new_clusters'] == cluster]) < i_min_cluster_size) and (cluster >= 0) and (min(df_beanup[df_beanup['new_clusters'] == cluster]['rate'])>0.0):
                    l_clusters_copy.remove(cluster)
            #
            # loop over beanup df and write clusters to list
            for i,row in df_beanup.iterrows():
                if row['new_clusters'] in l_clusters_copy:
                    l_filtered_clusters.append(row['new_clusters'])
                else:
                    l_filtered_clusters.append(-1)
            #   
        # end of loop over beanups to clean up small clusters
        #
        # add the filtered clusters to new column
        df_clustered['new_clusters_cleaned'] = l_filtered_clusters
        #
        ##################################
        # 4. Ensure bean-up monotonicity #
        ##################################
        #
        # last point is to loop over the bean-up and ensure monotonicity of ICV; also remove the noise points
        # methods that we have for ICV representationL min/ave/max ICV (recommend min ICV for monotonic increase, as if a cluster includes
        #   other ICV points with a less-than-optimal n_cluster/EPS then it should still monotonically increase on the next one)
        #
        # firstly, remove noise points from the df
        df_clustered = df_clustered[df_clustered['new_clusters_cleaned'] >= 0]
        #
        # define ICV measure
        i_icv_filter_measure = d_well_data[well][i_d_well_monotonicity_method]
        #
        # integer flags for filter measure
        i_icv_measure_min = 0   # for ICV filter method
        i_icv_measure_ave = 1   # for ICV filter method
        i_icv_measure_max = 2   # for ICV filter method
        i_icv_measure_med = 3   # for ICV filter method
        #
        # dictionary referencing
        i_cluster_min_ICV = 0           # dictionary ref for dictionary of monotonically increasing clusters
        i_cluster_start_slice = 1       # dictionary ref for dictionary of monotonically increasing clusters
        i_cluster_end_slice = 2         # dictionary ref for dictionary of monotonically increasing clusters
        i_cluster_monotonic_cluster = 3 # dictionary ref for dictionary of monotonically increasing clusters
        #
        # filtering behaviour on more than one monotonic increase
        i_icv_filter_behaviour = d_well_data[well][i_d_well_filter_behaviour]
        #
        # integer flags for filter behaviour
        i_cluster_takezero = 0     # logic comparison value
        i_cluster_takelongest = 1  # logic comparison value
        #
        # now loop over df and try to find the clusters that describe monotonic increase the best
        for i_df_cluster,beanup in enumerate(df_clustered['injector_period_ID'].unique()):
            #
            # loop list / counters
            d_cluster_monotonics = {}   # dictionary of monotonically increasing clusters (cluster:[icv,length])
            icv_previous = 0            # value of previous ICV (reset to zero)
            cluster_previous = -10      # value of cluster (reset to arbitrary value that doesn't reflect a DBSCAN/k-means number)
            #
            # get the beanup df
            df_beanup = df_clustered[df_clustered['injector_period_ID']==beanup].reset_index(drop=True)
            #
            # note that pandas returns unique values in order of appearence, which will be in 
            #   temporal order for us as we performed sort_values(by='datetime') at the beginning of the transform
            for cluster in df_beanup['new_clusters_cleaned'].unique():
                #
                # build dictionary entry - cluster:[ICV, start index, end index +1 (for Python slicing syntax ...)]
                if i_icv_filter_measure == i_icv_measure_min:
                    d_cluster_monotonics[cluster] = [min(df_beanup[df_beanup['new_clusters_cleaned']==cluster]['ICV']),df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[0],df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[-1]+1]
                elif i_icv_filter_measure == i_icv_measure_max:
                    d_cluster_monotonics[cluster] = [max(df_beanup[df_beanup['new_clusters_cleaned']==cluster]['ICV']),df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[0],df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[-1]+1]
                elif i_icv_filter_measure == i_icv_measure_ave:
                    d_cluster_monotonics[cluster] = [np.nanmean(df_beanup[df_beanup['new_clusters_cleaned']==cluster]['ICV']),df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[0],df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[-1]+1]
                else:
                    d_cluster_monotonics[cluster] = [np.nanmedian(df_beanup[df_beanup['new_clusters_cleaned']==cluster]['ICV']),df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[0],df_beanup[df_beanup['new_clusters_cleaned']==cluster].index.values[-1]+1]
                #
            #
            # loop variables for checking monotonicity
            my_icv = -99           # will refer to the ICV picked above that characterises each cluster
            i_cluster_loop = 0     # check number of 
            #
            # loop through cluster dictionary and append another value to the list corresponding to the monotonic increase "subcluster"
            #    (i.e. cluster on monotonicity, starting with 0 (i_cluster_loop))
            for i,cluster in enumerate(d_cluster_monotonics):
                #
                # if ICV is monotonically increasing, append the cluster-of-cluster value to list
                if d_cluster_monotonics[cluster][i_cluster_min_ICV] > my_icv: # note we are not taking the min ICV, this is just a handy reference to the 0th element in list (ICV)
                    d_cluster_monotonics[cluster].append(i_cluster_loop)      # add to the dictionary list above, i.e. list slice has gone from range(0,3) to range(0,4)
                    my_icv = d_cluster_monotonics[cluster][i_cluster_min_ICV]
                # if monotonicity of ICV is not observed, forward the monotonic subcluster counter and append this to list
                else:
                    i_cluster_loop += 1
                    d_cluster_monotonics[cluster].append(i_cluster_loop)
                    my_icv = d_cluster_monotonics[cluster][i_cluster_min_ICV]   
            #
            # make a df out of the monotonic subclustering dictionary to make life a little easier
            df_subcluster = pd.DataFrame.from_dict(d_cluster_monotonics,orient='index',columns=['icv','from_index','to_index','subcluster'])
            #
            # determine which cluster to take - initialise values
            i_mymax_cluster = 0   # initialise value of monotonic cluster prior to logic below
            #
            # now, define which clusters to take forward on the P(Q) plot based on monotonic filter behaviour specified in well dictionary
            if i_icv_filter_behaviour == i_cluster_takezero:
                # option 1 - take the one that starts at zero rate (should be the first)
                # (could improve this logic to look through all monotonic clusters for longest that starts at zero rate,
                #    but we already scrubbed the data so we are only looking at one continuous flowing period)
                i_mymax_cluster = df_subcluster['subcluster'].iloc[0]
            else:
                # loop over the clusters and work out how many rows are in each monotonic cluster, then take the longest
                d_cluster_sum = {}
                # first, generate a dictionary of montonoic cluster : number of df rows
                for ind,row in df_subcluster.iterrows():
                    i_cluster = row['subcluster']
                    if i_cluster in d_cluster_sum:
                        d_cluster_sum[i_cluster] += row['to_index']-row['from_index']
                    else:
                        d_cluster_sum[i_cluster] = row['to_index']-row['from_index']
                    #
                #
                # now loop over these and find the largest monotonic subcluster
                i_max_cluster = 0     # counter for total number of rows in cluster
                for cluster in d_cluster_sum:
                    if d_cluster_sum[cluster] > i_max_cluster:
                        i_mymax_cluster = cluster
                        i_max_cluster = d_cluster_sum[cluster]
                #
            # finally, get the slice integers (remember the "to" already has the Python +1 needed)
            i_slice_from = df_subcluster[df_subcluster['subcluster']==i_mymax_cluster]['from_index'].iloc[0]
            i_slice_to = df_subcluster[df_subcluster['subcluster']==i_mymax_cluster]['to_index'].iloc[-1]
            #
            # finally, slice the df to give a monotonically increasing P(Q) data set
            df_beanup = df_beanup.iloc[i_slice_from:i_slice_to]
            #
            # if we only have one cluster left, skip adding this cluster to the monotonic cluster list
            if len(df_beanup['new_clusters_cleaned'].unique()) < 2:
                continue
            #
            # build the final df
            if i_df_cluster == 0:
                df_clustered_monotonic = df_beanup.copy(deep=True)
            else:
                try:
                    df_clustered_monotonic = pd.concat([df_clustered_monotonic,df_beanup])
                except:
                    # above would fail if we removed the first bean-up from the data set and thus df_clustered_monotonic didn't exist, this should get around this
                    # (you could also just initialise df_clustered_monotonic with the df_beanup columns, but meh)
                    df_clustered_monotonic = df_beanup.copy(deep=True)
            #
        # end loop over bean-ups to enforce monotonicity of bean-up clusters
        #
        # print some stats
        print(str('Bean-up cleaning and clustering summary for well '+well+':'))
        print(str('  pre-filtering bean-ups: '+str(i_no_beanups)))
        print(str('  post-filtered, clustered and cleaned bean-ups: '+str(len(df_clustered_monotonic['injector_period_ID'].unique()))))
        print('')
        #
        # compile the interpreted well bean-ups
        if i_df == 0:
            df_final_beanup = df_clustered_monotonic.copy(deep=True) 
        else:
            try:
                df_final_beanup = pd.concat([df_final_beanup,df_clustered_monotonic],ignore_index=True)
            except:
                df_final_beanup = df_clustered.copy(deep=True) 
        #
    # end loop over each well
    #
    # drop/rename some columns that are no longer useful (e.g. injector period, they'll all be beanups here)
    df_final_beanup.drop('icv_int',axis=1,inplace=True)
    df_final_beanup.drop('injector_period_description',axis=1,inplace=True)
    df_final_beanup.drop('cluster',axis=1,inplace=True)
    df_final_beanup.drop('new_clusters',axis=1,inplace=True)
    df_final_beanup.rename(columns={'new_clusters_cleaned':'cluster'},inplace=True)
    #
    return df_final_beanup

In [ ]:
def beanup_QC(merged_beanup_clusters):
    #
    # short transform that goes through the bean-ups post-cleaning/clustering and
    #   gives a QC score that looks at number of steps, length per step etc.
    #
    #df = pd.concat([pick_clusters_DHG,pick_clusters_WHG],sort=False,ignore_index=True)
    df = merged_beanup_clusters.copy(deep=True)
    #
    # initialise df and list
    df_qc = pd.DataFrame(columns=['well','injector_period_ID','beanup_qc'])
    #
    for well in df['well'].unique():
        #
        # take sub-slice df of this well
        df_well = df[df['well']==well]
        #
        # loop over beanups and score between 0 (bad) and 1 (good)
        for beanup in df_well['injector_period_ID'].unique():
            #
            # reset QC score
            f_qc_score = 1.0
            #
            # get the sub-sliced df
            df_beanup = df_well[df_well['injector_period_ID']==beanup]
            #
            # adjust qc score for number of clusters (if > min clusters, don't penalise score, i.e. 1 - 1 is subtracted from QC score)
            f_qc_score -= 1.0 - min(1.0,(len(df_beanup['cluster'].unique())/i_min_clusters)**f_min_cluster_power)
            #
            # qc score for variance of cluster length (i.e. time spent within each cluster)
            #
            # build list of cluster lengths (note we cleaned up the noisy data, so each cluster should have value > -1)
            l_cluster_length = []   # list containing length of time at each cluster
            for cluster in df_beanup['cluster'].unique():
                l_cluster_length.append(len(df_beanup[df_beanup['cluster']==cluster]))
            #
            # measure the variability by the sample coefficient of variation for list, which is a nice normalised measure of dispersion (0 = perfect equivalent times at each step)
            #  (note this needs at least two clusters, otherwise np.std will divide by 0; cluster cleaning/filtering SHOULD guarantee this, but set hard penalty if it hasn't)
            try:
                f_CoV = (np.std(l_cluster_length,ddof=1)/np.mean(l_cluster_length))*f_CoV_penalty
            except:
                f_CoV = 1.0
            #
            # apply qc penalty
            f_qc_score -= f_CoV
            #
            # check qc is in bound [0,1]
            f_qc_score = min(max(f_qc_score,0.0),1.0)
            #
            # add data to df (remember double square brackets on list data, otherwise Pandas tries to make things nxn rather than 1xn size ...)
            df_qc = pd.concat([df_qc,pd.DataFrame([[well,beanup,f_qc_score]],columns=['well','injector_period_ID','beanup_qc'])],ignore_index=True)
            #
        # end loop over bean-ups
        print(str('Completed bean-up QC for well '+well))
    # end loop over wells 
    #
    # return the qc df
    return df_qc

In [ ]:
def beanup_PQ_stabilisation(merged_beanup_clusters):
    #
    # Transform that loops through the cleaned-up bean-up data and picks a single pressure and rate
    #   point per cluster / point of the bean-up
    #
    # This will allow us to do the final II / frac pressure picks, rather than using the whole data 
    #   set to do this
    #
    df = merged_beanup_clusters.copy(deep=True)
    #
    # initialise list to build final df from
    l_stab_points = []
    #
    for well in df['well'].unique():
        #
        # take sub-slice df of this well
        df_well = df[df['well']==well].copy(deep=True).sort_values(by='datetime')
        #
        # loop over beanups and derive P(Q) for each beanup
        for beanup in df_well['injector_period_ID'].unique():
            #
            # get beanup df to shorten typing required ...
            df_beanup = df_well[df_well['injector_period_ID']==beanup]
            #
            # (1) find the shortest cluster in the beanup and determine its length
            #
            i_shortest_cluster = 1000000
            for cluster in df_beanup['cluster'].unique():
                #
                # subslice df (makes the code more legible ...)
                df_cluster = df_beanup[df_beanup['cluster'] == cluster]
                #
                # skip update cluster length measure if rates are zero at this point
                if len(df_cluster[df_cluster['rate']<0.1]) > 0:
                    continue
                else:
                    if len(df_cluster) < i_shortest_cluster+1:
                        # recall that iloc slices on index, length will be index +1 ...
                        i_shortest_cluster = len(df_cluster)-1
                #
            #
            # if we still have the same value, just override with last value
            if i_shortest_cluster == 1000000:
                i_shortest_cluster = len(df_beanup['cluster'].unique()[-1]) - 1
            #
            # (2) determine the clip of the last beanup (as long as beanup has more than 2 clusters ...)
            #     -> this will make sure that a final bean-up step will not take into account long pressure build, i.e.
            #        we will clip the last stage of the bean-up based on the length of previous steps of the bean-up
            #
            if (len(df_beanup['cluster'].unique())>2):
                #
                i_sum_cluster = 0
                i_count_cluster = 0
                #
                for i_cluster,cluster in enumerate(df_beanup['cluster'].unique()):
                    #
                    # subslice df (makes the code more legible ...)
                    df_cluster = df_beanup[df_beanup['cluster'] == cluster]
                    #
                    # make sure this isn't the last cluster and isn't zero rate
                    if (i_cluster < len(df_beanup['cluster'].unique())-1) and (len(df_cluster[df_cluster['rate']<0.1]) == 0):
                        #
                        # count the number of points in the cluster
                        i_sum_cluster += len(df_cluster)-1
                        i_count_cluster += 1
                    #
                # determine the last cluster clip from mean of the cluster sizes
                try:
                    i_last_cluster_clip = int(i_sum_cluster/i_count_cluster)-2
                except:
                    i_last_cluster_clip = len(df_beanup[df_beanup['cluster'] == df_beanup['cluster'].unique()[-1]])-1
                #
            else:
                # just use the length of the last cluster otherwise
                i_last_cluster_clip = len(df_beanup[df_beanup['cluster'] == df_beanup['cluster'].unique()[-1]])-1
            #
            # finally, reset the last cluster clip based on its own size if needed
            i_last_cluster_clip = min(i_last_cluster_clip,len(df_beanup[df_beanup['cluster'] == df_beanup['cluster'].unique()[-1]])-1)
            #
            # (3)  loop over clusters and determine the stabilisation pick
            #
            for i_cluster,cluster in enumerate(df_beanup['cluster'].unique()):
                #
                # reset stabilisation values
                f_stab_p = 0.0
                f_stab_q = 0.0
                #
                # get df (again ... just keeps the line length down, should really just consider rolling up loops over bean-ups)
                df_cluster = df_beanup[df_beanup['cluster'] == cluster].reset_index(drop=True)
                #
                # for the representative temperature, just take the first temperature point rather than the average
                # (could add an option to control this, but I think it's a little overboard as really we care about initial and 
                #    frac onset temperature, which both happen at the "start" of a cluster)
                f_cluster_T = df_cluster['temperature'].iloc[0]
                #
                # if the cluster is the last one, we clip it to prevent a long pressure build (different transient) influencing P(Q) using the
                #    index derived above in (2)
                #
                if (i_cluster == len(df_beanup['cluster'].unique())-1):
                    #
                    # clip df (remember Python slices from:to-but-not-including!)
                    # (think I've doubled up and done -1+1 = 0 here, but I'm now too scared to get rid of them as it works)
                    df_cluster = df_cluster.iloc[0:i_last_cluster_clip+1]
                    #
                    # also override the shortest pick if necessary for i_stab_pick == i_stab_longestdt case
                    i_shortest_cluster = min(i_shortest_cluster,i_last_cluster_clip)
                    #
                #
                # if cluster has zero rate in it, we take the last point as its stabilisation regardless of method
                #
                # set up initial pressure point, we'll overwrite below
                f_stab_p0 = df_cluster['pressure'].iloc[0]
                #
                if len(df_cluster[df_cluster['rate']<1.0]) > 0:
                    #
                    # find the last point at zero rate
                    i_zero_rate = 0
                    for i,rate in enumerate(df_cluster['rate'].tolist()):
                        if rate < 1.0:
                            i_zero_rate = i
                        #
                    # return the p,q stabilisation point
                    f_stab_p1 = df_cluster['pressure'].tolist()[i_zero_rate]
                    f_stab_q1 = 0.0
                    f_stab_p2 = df_cluster['pressure'].tolist()[i_zero_rate]
                    f_stab_q2 = 0.0
                    f_stab_p3 = df_cluster['pressure'].tolist()[i_zero_rate]
                    f_stab_q3 = 0.0
                    f_stab_p4 = df_cluster['pressure'].tolist()[i_zero_rate]
                    f_stab_q4 = 0.0
                    f_stab_p5 = df_cluster['pressure'].tolist()[i_zero_rate]
                    f_stab_q5 = 0.0
                    #
                    # also record zero-point pressure here as we'll use this to condition the local stress state
                    f_stab_p0 = df_cluster['pressure'].tolist()[i_zero_rate]
                    #
                else:
                    #
                    # pick the stabilised pressure-rate point on the cross-plot for this cluster
                    #   (we'll record all methods and then select in the II determination)
                    #
                    # (1) take the last point in the cluster
                    f_stab_p1 = df_cluster['pressure'].iloc[-1]
                    f_stab_q1 = df_cluster['rate'].iloc[-1]
                    #
                    # (2) take the last point, but clip to 10-90 percentile of time (for choke change data)
                    clip_from = max(0,int(len(df_cluster)*0.1))
                    clip_to = min(len(df_cluster),int(len(df_cluster)*0.9))
                    #
                    # report back last point of the clipped series for rate and pressre
                    f_stab_p2 = df_cluster['pressure'].iloc[clip_from:clip_to].iloc[-1]
                    f_stab_q2 = df_cluster['rate'].iloc[clip_from:clip_to].iloc[-1]
                    #
                    # (3) take the point at which (dP/dt,dQ/dt) is at a minimum
                    #
                    # compute the derivative values
                    # we can cheat here as the delta time is the same between each row thanks to interpolation (probably)
                    #  (I could be more thorough and build in a Timedelta column to calculate the actual dt, but meh)
                    l_dp_dt = df_cluster['pressure'].diff().iloc[1:len(df_cluster)].abs()
                    l_dq_dt = df_cluster['rate'].diff().iloc[1:len(df_cluster)].abs()
                    #
                    # sum the two series (take absolute value - done above) and then do a simple lookup to find the stab point
                    l_sum_dt = l_dp_dt.add(l_dq_dt,fill_value=10000000.0)
                    i_stab_point = l_sum_dt.loc[l_sum_dt==min(l_sum_dt)].index.values[0]  # should only return a single value ... (i.e. should pick first if we have two identical minima)
                    #
                    # report back the stabilisation point
                    f_stab_p3 = df_cluster['pressure'].iloc[i_stab_point]
                    f_stab_q3 = df_cluster['rate'].iloc[i_stab_point]
                    #
                    # (4) take the point at which (dP/dt,dQ/dt) is at a minimum, with P10-90 clip applied
                    # same as above, but clip to P10-90 of the cluster
                    #
                    # compute the derivative values
                    # we can cheat here as the delta time is the same between each row thanks to interpolation
                    #
                    clip_from = max(1,int(len(df_cluster)*0.1))             # P10 clip extent (overwrite 0 with 1 here so we don't include the first NaN in diff() series)
                    clip_to = min(len(df_cluster),int(len(df_cluster)*0.9)) # P90 clip extent
                    l_dp_dt = df_cluster['pressure'].diff().iloc[clip_from:clip_to].abs()
                    l_dq_dt = df_cluster['rate'].diff().iloc[clip_from:clip_to].abs()
                    #
                    # sum the two series (take absolute value!) and then do a simple lookup to find the stab point
                    l_sum_dt = l_dp_dt.add(l_dq_dt,fill_value=10000000.0)
                    i_stab_point = l_sum_dt.loc[l_sum_dt==min(l_sum_dt)].index.values[0]  # should be OK for a single value ...
                    #
                    # report back the stabilisation point
                    f_stab_p4 = df_cluster['pressure'].iloc[i_stab_point]
                    f_stab_q4 = df_cluster['rate'].iloc[i_stab_point]
                    #
                    # (5) take consistent dt transient
                    # report back the stabilisation point at the same dt (based on shortest cluster)
                    f_stab_p5 = df_cluster['pressure'].iloc[i_shortest_cluster]
                    f_stab_q5 = df_cluster['rate'].iloc[i_shortest_cluster]
                # (end if statement if cluster has zero rate)
                #
                # build df list - we'll have a row per stabilisation type so all data is available for debugging or 
                #   choosing the best stabilisation method for the well
                l_stab_points.append([well,beanup,cluster,i_stab_last,f_stab_p0,f_stab_p1,f_stab_q1,f_cluster_T])
                l_stab_points.append([well,beanup,cluster,i_stab_lastclip,f_stab_p0,f_stab_p2,f_stab_q2,f_cluster_T])
                l_stab_points.append([well,beanup,cluster,i_stab_minderv,f_stab_p0,f_stab_p3,f_stab_q3,f_cluster_T])
                l_stab_points.append([well,beanup,cluster,i_stab_mindervclip,f_stab_p0,f_stab_p4,f_stab_q4,f_cluster_T])
                l_stab_points.append([well,beanup,cluster,i_stab_longestdt,f_stab_p0,f_stab_p5,f_stab_q5,f_cluster_T])
            # (end loop over clusters)
        # (end loop over beanups)
        print(str('Completed beanup p-q picks for well '+well))
    # (end loop over wells)
    #
    # build df from detected stabilisation points
    df_pq = pd.DataFrame(l_stab_points,columns=['well','injector_period_ID','cluster','stab_type','pressure_initial','pressure','rate','temperature'])
    #
    return df_pq

In [ ]:
def beanup_II_tracker(beanup_PQ_stabilisation):
    #
    # Transform that loops through the P(Q) picks for each bean-up and derives
    #   injectivity index, frac pressure etc. for the well
    #
    # Loops over each bean-up and assigns point to II clusters based on local dP/dQ gradient
    # Methods used: (1) simple averaging over II clusters, (2) linear regression over II clusters, (3) piecewise linear regression over all data
    #
    df = beanup_PQ_stabilisation.copy(deep=True)
    #
    # set up final injectivity df
    df_injectivity = pd.DataFrame(columns=['well','injector_period_ID','II_measure','flow_regime','II_field','II_metric','intercept_field','intercept_metric','Pfrac_field','Pfrac_metric','Qfrac_field','Qfrac_metric'])
    #
    for i_df,well in enumerate(df['well'].unique()):
        #
        # take sub-slice df of this well
        df_well = df[df['well']==well]
        #
        # loop over beanups and derive injectivity and frac pressure for each beanup
        for beanup in df_well['injector_period_ID'].unique(): 
            #
            # get the beanup df
            df_beanup = df_well[df_well['injector_period_ID']==beanup].copy(deep=True)#.reset_index(drop=True)
            #
            # sub-slice this df based on P(Q) stabilisation method (reset the index so we can do proper calling of df indices 0,1,2,...,n)
            df_beanup = df_beanup[df_beanup['stab_type'] == d_well_data[well][i_d_well_stabilisation_method]].reset_index(drop=True)
            #
            # check that we have a unique cluster data set, warn if we don't
            if len(df_beanup) != len(df_beanup['cluster'].unique()):
                print(str('Warning: cluster set for bean-up II derivation is non-unique ... continuing with caution'))
                print(df_beanup.head(10))
            #
            ############################################################################################
            # (1) calculate injectivity with II clustering (don't use frac pressures, derive a priori) #
            ############################################################################################
            #
            # calculate the raw dp/dt values
            l_ii_grad = []
            q_prev = 0.0
            p_prev = 0.0
            q_0 = df_beanup['rate'].iloc[0]
            p_0 = df_beanup['pressure'].iloc[0]
            #
            # append II values to the beanup df (remember - this is the inverse of the P(Q) plot gradient)
            for i,r in df_beanup.iterrows():
                #
                try:
                    # ignore first row data - np.nan will be ignored by cluster data
                    if i == 0:
                        l_ii_grad.append(np.nan)
                    #
                    # otherwise generate the guess of II for this point (we'll guess field-unit II, which is actaully the inverse of the PQ plot)
                    else:
                        # method 1, use immediate first-order backwards derivative
                        if d_well_data[well][i_d_well_dbscan_discretisation] == 0:
                            l_ii_grad.append(((r['rate'] - q_prev)/(r['pressure'] - p_prev))*f_II_metric_to_field)
                        #
                        # method 2, use total first-order backwards derivative
                        elif d_well_data[well][i_d_well_dbscan_discretisation] == 1:
                            l_ii_grad.append(((r['rate'] - q_0)/(r['pressure'] - p_0))*f_II_metric_to_field)
                        #
                        # method 3, use instantaneous P/Q (you mad man/woman)
                        else:
                            l_ii_grad.append((r['rate']/r['pressure'])*f_II_metric_to_field)
                except:
                    l_ii_grad.append(0.0)
                #
                # update previous guesses
                q_prev = r['rate']
                p_prev = r['pressure']
            #
            # append the estimated II to the beanup df
            df_beanup.loc[:,'ii_gradient'] = l_ii_grad
            #
            ##############################################################
            # (2) use DBSCAN to generate clusters of similar injectivity #
            ##############################################################
            #
            # Set up the epsilon parameter. This controls max distance to be considered in same centroid;
            #    typically frac mode is >5-10 stb/d/psi different so use this as a starting point.
            #    - you'll want to do some sensitivity study for each well to see II(t,EPS)
            #    - also bear in mind we may have initially high II on long shut in bean ups (higher BHT > lower viscosity water); it's the DIFFERENCE between regimes we care about
            #    => too high a value = everything is considered to be the same flow  regime
            #    => too low a value = everything is considered to be in a different flow regime
            f_dbscan_eps = d_well_data[well][i_d_well_dbscan_II_eps]
            f_dbscan_minsamples = d_well_data[well][i_d_well_dbscan_II_minsamples]
            #
            # find and store rows with NaN in II (won't use these to cluster, but we'll add them back in at the end)
            # these could be (1) first row values (backwards II from above will have NaN for first value) (2) values where P is identical between subsequent points
            my_nan_indices = []
            my_nan_rows = []
            for i,r in df_beanup.iterrows():
                if np.isnan(r['ii_gradient']):
                    my_nan_indices.append(i)
                    my_nan_rows.append(r.values)
            #
            # drop these rows from the df
            df_beanup = df_beanup.iloc[[i for i in df_beanup.index.tolist() if i not in my_nan_indices]]
            #
            # get the 1D data (II) to cluster on and get it into np array so we can run DBSCAN
            cluster_data = np.array(df_beanup['ii_gradient'].dropna())     # the .iloc[] logic above SHOULD remove all NaNs, so the dropna() is just in case
            cluster_data = np.reshape(cluster_data,(len(cluster_data),1))  # DBSCAN wants an exactly 1D array for input ...
            #
            # generate the clusters
            dbscan_cluster = DBSCAN(eps=f_dbscan_eps,min_samples=f_dbscan_minsamples).fit(cluster_data)
            #
            # build the cluster list, note first data point doesn't have II computed so label as noise (-1 in cluster parlance)
            l_dbscan_cluster = dbscan_cluster.labels_.tolist()
            #
            # update cluster labelling to include any points of the II gradient with no cluster value (usually the first point)
            for index in my_nan_indices:
                try:
                    # insert the value and use cluster label that is the next point in the list
                    l_dbscan_cluster.insert(index,l_dbscan_cluster[index])
                except:
                    # if this fails, index must be greater than the df, so use the value of the last one and insert at the end
                    l_dbscan_cluster.insert(len(l_dbscan_cluster),l_dbscan_cluster[-1])
            #
            # add the nan rows back in to the beanup df
            df_beanup = pd.concat([df_beanup,pd.DataFrame(my_nan_rows,index=my_nan_indices,columns=df_beanup.columns)],sort=False).sort_index()
            #
            # if we still have missing data, add more noise values
            # (the logic above SHOULD plug these holes, so this SHOULD be unnecessary ... but never say never)
            if len(l_dbscan_cluster) < len(df_beanup):
                #
                # number of missing data
                i_dbscan_missing = len(df_beanup) - len(l_dbscan_cluster)
                #
                # append missing data
                for i in range(0,i_dbscan_missing):
                    l_dbscan_cluster.insert(0,-1)
            #
            # (or take them away)
            elif len(df_beanup) < len(l_dbscan_cluster):
                l_dbscan_cluster = l_dbscan_cluster[0:len(df_beanup)]
            #
            # append the cluster labels to the beanup df
            try:
                df_beanup.loc[:,'ii_cluster_labels'] = l_dbscan_cluster
            except:
                print('error at: ')
                print(well,beanup)
                print(len(df_beanup),len(l_dbscan_cluster))
                df_beanup.loc[:,'ii_cluster_labels'] = l_dbscan_cluster
            #
            # finally, go through the clusters and merge clusters who have a lower II in the next cluster (if requested)
            if i_want_increasing_II == 1:
                #
                # set up loop variables
                my_ave_II = 0.0
                my_prev_cluster = df_beanup['ii_cluster_labels'].iloc[0]
                l_new_clusters = []
                #
                # loop over each cluster, determine average II and merge clusters if average II is less than previous one
                for cluster in df_beanup['ii_cluster_labels'].unique():
                    #
                    f_ave_cluster_II = np.nanmean(df_beanup[df_beanup['ii_cluster_labels'] == cluster]['ii_gradient'])
                    #
                    if f_ave_cluster_II < my_ave_II:
                        # if average II is less than before, use previous cluster number
                        l_new_clusters += [my_prev_cluster]*len(df_beanup[df_beanup['ii_cluster_labels'] == cluster])
                    else:
                        my_prev_cluster = cluster   # update previous cluster to this one
                        my_ave_II = f_ave_cluster_II  # update threshold II to merge cluster
                        l_new_clusters += [cluster]*len(df_beanup[df_beanup['ii_cluster_labels'] == cluster])
                    #
                #
                # finally, re-write cluster labels with new ones
                try:
                    df_beanup.loc[:,'ii_cluster_labels'] = l_new_clusters
                except:
                    print(len(df_beanup['ii_cluster_labels']),len(l_new_clusters))
                    df_beanup.loc[:,'ii_cluster_labels'] = l_new_clusters
            #
            #
            #########################################################################
            # (3) injectivity measure 1 - averaged II numbers over each cluster     #
            # (4) injectivity measure 2 - perform linear regression on each cluster #
            #########################################################################
            #
            # set up an empty list to build data for injectivity df
            l_dfinj_rows = []
            #
            # set up loop parameters
            my_regime = 1                     # updates the picked flow regime
            i_prev_counter = 0                # counts number of flow regimes prior to the one under consideration
            my_prev_metric_II = 0.0           # previous metric II, averaged value (to compute frac pressure)
            my_prev_metric_int = 0.0          # previous metric intercept, averaged value (to compute frac pressure)
            my_prev_metric_reg_II = 0.0       # previous metric II, linearly regressed value (to compute frac pressure)
            my_prev_metric_reg_int = 0.0      # previous metric intercept, linearly regressed value (to compute frac pressure)
            #
            # compute linear regressed and averaged injectivities for each cluster (~ flow regime)
            for i_cluster,cluster in enumerate(df_beanup['ii_cluster_labels'].unique()):
                #
                # grab df for just this II cluster
                df_cluster = df_beanup[df_beanup['ii_cluster_labels']==cluster]
                #
                # if this is not the first cluster, also add the previous row of pressure/rate data back into the cluster df
                # (as we clustered II grouping on backwards-differenced derivatives, so previous value was used for this calc!)
                if (i_cluster > 0) and ((len(df_cluster) < i_add_previous_row) or (i_add_previous_row == 0)):
                    #
                    # index to pull is i-1 in the original beanup df
                    i_pull_index = (df_cluster.index[0]) - 1
                    #
                    if i_pull_index > 0:
                        df_cluster = pd.concat([df_cluster,pd.DataFrame([df_beanup.iloc[df_cluster.index[0]-1].values],columns=df_cluster.columns)],sort=False).sort_index()
                #
                # reset the df_cluster indices for subsequent integer slicing
                df_cluster.reset_index(drop=True,inplace=True)
                #
                if cluster >= 0: # don't write regime for noise (DBSCAN labels noise as -1)
                    # 
                    # general df options
                    flow_regime = str('regime_'+str(my_regime))   # label all regimes as regime_1, regime_2 etc.
                    II_ave = -1.0                                 # set up token averaged II value to begin
                    #
                    # compute the averaged injectivity (np.nanmean() will ignore NaNs)
                    f_ave_field_II = np.nanmean(df_cluster['ii_gradient'])
                    f_ave_metric_II = f_ave_field_II/f_II_metric_to_field
                    #
                    # compute intercept from averaged injectivity (from last point in cluster) - remember true gradient = 1/II
                    f_ave_metric_intercept = (df_cluster['pressure'].iloc[-1])-((1.0/f_ave_metric_II)*df_cluster['rate'].iloc[-1])
                    f_ave_field_intercept = f_ave_metric_intercept*f_bar_to_psi
                    #
                    # set up 1D rate/pressure arrays for linear regression (have to reshape the arrays to be 1D column vectors)
                    rate_data = np.array(df_cluster['rate'])
                    rate_data = np.reshape(rate_data,(len(rate_data),1))
                    pressure_data = np.array(df_cluster['pressure'])
                    pressure_data = np.reshape(pressure_data,(len(pressure_data),1))
                    #
                    # perform linear regression on P(Q) points and save linear regression coefficients
                    try:
                        #
                        lin_reg = LinearRegression().fit(rate_data,pressure_data)
                        # 
                        f_lin_reg_metric_II = 1.0 / lin_reg.coef_[0][0]
                        f_lin_reg_field_II = f_lin_reg_metric_II*f_II_metric_to_field
                        f_lin_reg_metric_intercept = lin_reg.intercept_[0]
                        f_lin_reg_field_intercept = f_lin_reg_metric_intercept*f_bar_to_psi
                    #
                    # if linear regression fails (e.g. only one point), write token noise values
                    except:
                        f_lin_reg_metric_II = -1.0
                        f_lin_reg_field_II = -1.0
                        f_lin_reg_metric_intercept = -1.0
                        f_lin_reg_field_intercept = -1.0
                    #
                    # compute frac pressure / breakover (only if there is a previous cluster)
                    # then append the data to the l_dfinj_rows list (to be later appended to df_injectivity)
                    if i_prev_counter > 0:
                        #
                        # averaged values over each cluster
                        q_breakover_ave_metric = (f_ave_metric_intercept-my_prev_metric_int)/((1.0/my_prev_metric_II)-(1.0/f_ave_metric_II))
                        q_breakover_ave_field = q_breakover_ave_metric*f_m3hr_to_bpd
                        p_breakover_ave_metric = f_ave_metric_intercept + (1.0/f_ave_metric_II)*q_breakover_ave_metric
                        p_breakover_ave_field = p_breakover_ave_metric*f_bar_to_psi
                        #
                        # linear regression values over each cluster
                        q_breakover_reg_metric = (f_lin_reg_metric_intercept-my_prev_metric_reg_int)/((1.0/my_prev_metric_reg_II)-(1.0/f_lin_reg_metric_II))
                        q_breakover_reg_field = q_breakover_reg_metric*f_m3hr_to_bpd
                        p_breakover_reg_metric = f_lin_reg_metric_intercept + (1.0/f_lin_reg_metric_II)*q_breakover_reg_metric
                        p_breakover_reg_field = p_breakover_reg_metric*f_bar_to_psi
                        #
                        # store average values into list-of-lists
                        l_dfinj_rows.append([well,
                                            beanup,
                                            'cluster_averaged_values',
                                            flow_regime,
                                            f_ave_field_II,
                                            f_ave_metric_II,
                                            f_ave_field_intercept,
                                            f_ave_metric_intercept,
                                            p_breakover_ave_field,
                                            p_breakover_ave_metric,
                                            q_breakover_ave_field,
                                            q_breakover_ave_metric])
                        #
                        # store regressed values into list-of-lists
                        l_dfinj_rows.append([well,
                                            beanup,
                                            'cluster_lin_regressed_values',
                                            flow_regime,
                                            f_lin_reg_field_II,
                                            f_lin_reg_metric_II,
                                            f_lin_reg_field_intercept,
                                            f_lin_reg_metric_intercept,
                                            p_breakover_reg_field,
                                            p_breakover_reg_metric,
                                            q_breakover_reg_field,
                                            q_breakover_reg_metric])
                        #
                    #
                    # if first regime, write average values with some token breakover values (my convention is negative = null data, rather than np.nan)
                    else:
                        l_dfinj_rows.append([well,
                                            beanup,
                                            'cluster_averaged_values',
                                            flow_regime,
                                            f_ave_field_II,
                                            f_ave_metric_II,
                                            f_ave_field_intercept,
                                            f_ave_metric_intercept,
                                            -1.0,
                                            -1.0,
                                            -1.0,
                                            -1.0])
                        #
                        # store regressed values
                        l_dfinj_rows.append([well,
                                            beanup,
                                            'cluster_lin_regressed_values',
                                            flow_regime,
                                            f_lin_reg_field_II,
                                            f_lin_reg_metric_II,
                                            f_lin_reg_field_intercept,
                                            f_lin_reg_metric_intercept,
                                            -1.0,
                                            -1.0,
                                            -1.0,
                                            -1.0])
                    #
                    # march forward regime
                    my_regime += 1
                    #
                    # march cluster counter (could actually just combine this with above, but why break an otherwise working code)
                    i_prev_counter += 1
                    #
                    # save regression outputs of this cluster in case there is another regime
                    my_prev_metric_II = f_ave_metric_II
                    my_prev_metric_int = f_ave_metric_intercept
                    my_prev_metric_reg_II = f_lin_reg_metric_II
                    my_prev_metric_reg_int = f_lin_reg_metric_intercept
                    #
                # end of check cluster for noise
            # end of loop over cluster for linear/averaged II
            #
            ####################################################################
            # (5) injectivity measure 3 - a priori piecewise linear regression #
            ####################################################################
            #
            # (this is the serious stuff ... ish)
            #
            # firstly, filter out noise so we are just doing piecewise regression on identified regimes
            # (note that this can result in harsh filtering of single-point regimes; we can get around this by setting 
            #    the i_d_well_dbscan_II_minsamples values in the well dictionary to 1)
            df_pwlf_temp = df_beanup[df_beanup['ii_cluster_labels']>=0]
            #
            # if requested, go through the df and add the last point from each cluster back into a new cluster
            # (i.e. we did a backwards difference, so double up on the knot points for PWLF)
            if i_double_knots == 1:
                #
                my_ii_cluster = df_pwlf_temp['ii_cluster_labels'].iloc[0]
                my_old_row = df_pwlf_temp.iloc[0].values.tolist()
                l_new_pwlf_data = []
                #
                for i_df,row_df in df_pwlf_temp.iterrows():
                    #
                    # if new cluster, re-append the old (last) row as an extra row (change ii_cluster_labels to new cluster)
                    if row_df['ii_cluster_labels'] != my_ii_cluster:
                        my_old_row[len(my_old_row)-1] = row_df['ii_cluster_labels']        
                        l_new_pwlf_data.append(my_old_row)
                    #
                    # add the current row
                    l_new_pwlf_data.append(row_df.values.tolist())
                    #
                    my_old_row = row_df.values.tolist()
                    my_ii_cluster = row_df['ii_cluster_labels']
                #
                # re-write the df
                df_pwlf = pd.DataFrame(l_new_pwlf_data,columns=df_pwlf_temp.columns)
            else:
                # otherwise just use current pulled df
                df_pwlf = df_pwlf_temp
            #
            # get number of linear/knotted sections (this will feed piecewise algorithm used, otherwise we may over-fit)
            # (i.e. the best fit piecewise linear function will have N-1 sections for N data points ... d'oh)
            i_pwlf_sections = len(df_pwlf['ii_cluster_labels'].unique())
            #
            # (could add something here that if len(df_pwlf) = 3 and i_pwlf_sections = 2 then duplicate the middle point as a 
            #    human would effectively do to fit two straight lines; for now use the i_double_knots global integer flag instead)
            #
            # if we don't have enough points (need npoints >= 2*nsections to constrain), try reducing the number of PWLF sections;
            i_pop = 0
            if (len(df_pwlf) < 2*i_pwlf_sections) and (i_pwlf_sections > 1): 
                i_pwlf_sections -= 1
                i_pop += 1
                # double this up for added fudging in case we have e.g. 3 points, 3 II clusters / sections
                if (len(df_pwlf) < 2*i_pwlf_sections) and (i_pwlf_sections > 1): 
                    i_pwlf_sections -= 1       
                    i_pop += 1
            #        
            # build the list of guessed breakover points if we have more than one linear section 
            # (we'll use this to initialise the piecewise solver rather than give it generic guesses)
            if i_pwlf_sections > 1:
                #
                # initialise list of breakover rate points
                l_breakover_q = []
                # 
                # grab first cluster details
                l_cluster_label = [df_pwlf['ii_cluster_labels'].iloc[0]]
                #
                # initialise previous rate (used to derive mid point where the cluster changes)
                f_prev_rate = df_pwlf['rate'].iloc[0]
                #
                # loop over rows and find point where new cluster begins; where it begins, record the mid-point rate (likely piecewise function breakover point)
                i_breakover_q_points = 0 # ensure we have the right number of initial guesses with a counter, in case we reduced i_pwlf_sections above
                for i,row in df_pwlf.iterrows():
                    #
                    # if the cluster in the current row is new, assign mid-point guess to the breakover rate list
                    if (row['ii_cluster_labels'] not in l_cluster_label) and (i_breakover_q_points < i_pwlf_sections - 1):
                        #
                        # if new cluster row, append average rate over the cluster change
                        l_breakover_q.append(0.5*(row['rate']+f_prev_rate))
                        # store cluster label so we don't record this one again
                        l_cluster_label.append(row['ii_cluster_labels'])
                        # increment the breakpoint counter
                        i_breakover_q_points += 1
                        #
                    #
                    # update previous rate to store value from this row (so it's always "live")
                    f_prev_rate = row['rate']
                #
                # check number of breakover points are correct (could also modify this to clip or add as required ...)
                if len(l_breakover_q) != i_pwlf_sections - 1:
                    print('Warning: incorrect number of initial rate guesses for piecewise linear regression on cluster data')
                    print('Well = ',well,' Beanup = ',beanup)
                    print('Number of linear sections: ',i_pwlf_sections - 1)
                    print('Breakover rate guesses: ',len(l_breakover_q))
                #
                # finally, loop over clusters and estimate injectivity from average values
                # (should map correctly to breakover points above ...!)
                l_cluster_II = []
                i_breakover_II_points = 0 # ensure we have the right number of initial guesses with a counter, in case we reduced i_pwlf_sections above
                for cluster in df_pwlf['ii_cluster_labels'].unique():
                    if (cluster >= 0) and (i_breakover_II_points < i_pwlf_sections):
                        l_cluster_II.append(np.nanmean(df_pwlf[df_pwlf['ii_cluster_labels']==cluster]['ii_gradient']))
                        i_breakover_II_points += 1
                #
                # similarly, check number of II guesses
                if len(l_cluster_II) != i_pwlf_sections:
                    print('Warning: incorrect number of initial injectivity index guesses for piecewise linear regression on cluster data')
                    print('Well = ',well,' Beanup = ',beanup)
                    print('Breakover sections: ',i_pwlf_sections - 1)
                    print('II guesses: ',len(l_cluster_II))
                #
            #
            # now, we'll set up piecewise linear function (can currently cope with 2, 3 or 4; if >4 then I'm assuming this isn't right)
            # -> this ain't pretty but I'm not sure how to parametise this within np/sp
            #
            # also set up the initial estimates for a function with N linear parts:
            # x0, x1, x2 are piecewise knots (breakover points, x < x0, x0 <= x < x1, ... - have N-1 values)
            # b is the initial intercept (you can derive other intercepts from this, only have 1 value)
            # k1, k2, k3 are gradients of each section (have N values, I've confusingly started from 1 rather than 0 to annoy and I'm too frightened to break code to change)
            #
            # only set up the piecewise function if we have more than one linear section 
            # (otherwise we just do a simple linear regression and there's no difference to above)
            if i_pwlf_sections == 2:
                #
                # piecewise linear function definition
                def f_pwlf(x,x0,y0,k1,k2):
                    return np.piecewise(x, [x < x0, x >= x0], [lambda x:k1*x + y0, lambda x:k2*x + (k1-k2)*x0 + y0])
                #
                # guessed parameters
                p_x0 = l_breakover_q[0]                # breakover 1 rate
                p_y0  = df_beanup['pressure'].iloc[0]  # pressure at zero/first rate (don't ignore noise data here, use beanup df)
                p_k1 = f_II_metric_to_field / l_cluster_II[0]   # gradient of plot, bar/m3/hr
                p_k2 = f_II_metric_to_field / l_cluster_II[1]   #   (remember this is 1/II, also need to convert back to metric!)
                #
                # build guessed arguments list
                p0 = [p_x0,p_y0,p_k1,p_k2]
                #
            elif i_pwlf_sections == 3:
                #
                # piecewise linear function definition
                def f_pwlf(x,x0,x1,y0,k1,k2,k3):
                    return np.piecewise(x, [x < x0, (x >= x0)&(x < x1), x >= x1], [lambda x:k1*x + y0, lambda x:k2*x + (k1-k2)*x0 + y0, lambda x:k3*x + (k2-k3)*x1 + (k1-k2)*x0 + y0])
                #
                # guessed parameters
                p_x0 = l_breakover_q[0]                # breakover 1 rate
                p_x1 = l_breakover_q[1]                # breakover 2 rate
                p_y0  = df_beanup['pressure'].iloc[0]  # pressure at zero/first rate (don't ignore noise data here, use beanup df)
                p_k1 = f_II_metric_to_field / l_cluster_II[0]   # gradient of plot, bar/m3/hr
                p_k2 = f_II_metric_to_field / l_cluster_II[1]   #   (remember this is 1/II, also need to convert back to metric!)
                p_k3 = f_II_metric_to_field / l_cluster_II[2]   #   
                #
                # build guessed arguments list
                p0 = [p_x0,p_x1,p_y0,p_k1,p_k2,p_k3]
                #
            elif i_pwlf_sections > 3:
                #
                # piecewise linear function definition
                def f_pwlf(x,x0,x1,x2,y0,k1,k2,k3,k4):
                    return np.piecewise(x, [x < x0, (x >= x0)&(x < x1), x >= x1], [lambda x:k1*x + y0, lambda x:k2*x + (k1-k2)*x0 + y0, lambda x:k3*x + (k2-k3)*x1 + (k1-k2)*x0 + y0, lambda x:k4*x + (k3-k4)*x2 + (k2-k3)*x1 + (k1-k2)*x0 + y0])
                #
                # guessed parameters
                p_x0 = l_breakover_q[0]                # breakover 1 rate
                p_x1 = l_breakover_q[1]                # breakover 2 rate
                p_x2 = l_breakover_q[2]                # breakover 3 rate
                p_y0  = df_beanup['pressure'].iloc[0]  # pressure at zero rate (don't ignore noise data here, use beanup df)
                p_k1 = f_II_metric_to_field / l_cluster_II[0]   # gradient of plot, bar/m3/hr
                p_k2 = f_II_metric_to_field / l_cluster_II[1]   #   (remember this is 1/II, also need to convert back to metric!)
                p_k3 = f_II_metric_to_field / l_cluster_II[2]   #   
                p_k4 = f_II_metric_to_field / l_cluster_II[3]   #   
                #
                # build guessed arguments list
                p0 = [p_x0,p_x1,p_x2,p_y0,p_k1,p_k2,p_k3,p_k4]
                #
            #
            # get the rate/pressure data for the piecewise regression (don't need reshaped arrays for this as scipy is smart)
            rate_data = np.array(df_pwlf['rate'])
            pressure_data = np.array(df_pwlf['pressure'])
            #
            # perform (piecewise) linear regression and store the II data
            #
            if i_pwlf_sections == 1: # one section -> just do normal linear regression (get the same answer as above)
                #
                # this time we have to re-shape rate/pressure data for linear regression (needs to be 1D np array)
                rate_data = np.reshape(rate_data,(len(rate_data),1))
                pressure_data = np.reshape(pressure_data,(len(pressure_data),1))
                #    
                try:
                    #
                    lin_reg = LinearRegression().fit(rate_data,pressure_data)
                    #
                    # determine II and intercept from the coefficients column vector
                    f_II_metric = 1.0 / lin_reg.coef_[0][0]
                    f_II_field = f_II_metric * f_II_metric_to_field
                    #
                    f_int_metric = lin_reg.intercept_[0]
                    f_int_field = f_int_metric * f_bar_to_psi
                #
                except:
                    # if regression fails, use token values
                    f_II_metric = -1
                    f_II_field = -1
                    #
                    f_int_metric = -1
                    f_int_field = -1
                #
                # store injectivity values into list to add to injectivity df
                l_dfinj_rows.append([
                    well,
                    beanup,
                    'pwlf_interp_values',
                    'regime_1',
                    f_II_field,
                    f_II_metric,
                    f_int_field,
                    f_int_metric,
                    -1.0,
                    -1.0,
                    -1.0,
                    -1.0
                ])
                #
            elif i_pwlf_sections > 1:
                #
                # run scipy optimiser to take initial guesses and update coefficients (in "p")
                #
                # note this will fail if number of data points < number of unknowns (which happens regularly for GL chokes!), so we'll wrap this around a try-except logic
                # (I added some logic above to duplicate points and/or shrink number of piecewise sections to try to avoid this outcome)
                try:
                    #
                    # p = list of the optimised parameters, e = estimated covariance of the optimised parameters
                    #
                    # we'll use default LS optimisation (Levenberg-Marquardt), fall back to more robust method if it fails
                    p,e = optimize.curve_fit(f_pwlf,rate_data,pressure_data,p0)
                    #
                    # store injectivity values into list to add to injectivity df
                    # -> for more than one regime, we also have breakovers to determine
                    # -> also remember intercepts are not explicitly stored in optimize's p list! we need to back extract these
                    #
                    # first, let's pull gradient and intercept lists from the p (regressed) list
                    l_xn = []
                    l_kn = []
                    #
                    # reminder: list of parameters is [x0,x1,...],y0,[k1,k2,...]
                    for i in range(0,i_pwlf_sections):
                        #
                        # get the x intercepts (remember 1 fewer than total sections)
                        if i < (i_pwlf_sections-1) :
                            l_xn.append(p[i])
                        #
                        # get the k/gradient values (equal number to total sections, located in array after the xn/y0 values)
                        l_kn.append(p[i_pwlf_sections+i])
                        #
                    # get the y0 value (always a single value, remember Python list indexing starts at 0)
                    f_y0 = p[i_pwlf_sections-1]
                    #
                    # back out the intercepts for each section
                    l_intercepts = [f_y0] # first linear section is explicitly defined as y0
                    #
                    # other intercepts are defined one on from the other, so have to back-calculate them in each piecewise linear section
                    for i in range(1,i_pwlf_sections):
                        l_intercepts.append(l_intercepts[i-1] + ((l_kn[i-1]-l_kn[i])*l_xn[i-1]))
                    #
                    # now loop over each linear section (flow regime) and add data to the injectivity list for the df build
                    for i in range(0,i_pwlf_sections):
                        #
                        # generate the regime name
                        flow_regime = str('regime_'+str(i+1))
                        #
                        # get the injectivity index
                        f_metric_II = 1.0 / l_kn[i]
                        f_field_II = f_metric_II * f_II_metric_to_field
                        #
                        # get the intercept
                        f_metric_int = l_intercepts[i]
                        f_field_int = f_metric_int * f_bar_to_psi
                        #
                        # get the crossover pressure (note x-over rate is already pulled from the l_xn list)
                        if i == 0:
                            #
                            # use token -1 values for first regime
                            q_breakover_metric = -1.0
                            q_breakover_field = -1.0
                            p_breakover_metric = -1.0
                            p_breakover_field = -1.0
                            #
                            # update the previous values
                            f_breakover_int_prev = l_intercepts[i]
                            f_breakover_grad_prev = l_kn[i]
                            #
                        else:
                            #
                            # generate the q/p breakover points
                            q_breakover_metric = l_xn[i-1]
                            p_breakover_metric = l_intercepts[i] + (l_kn[i]*q_breakover_metric)
                            #
                            # convert to field units
                            q_breakover_field = q_breakover_metric * f_m3hr_to_bpd
                            p_breakover_field = p_breakover_metric * f_bar_to_psi
                            #
                        #
                        # append data to injectivity list build
                        l_dfinj_rows.append([
                            well,
                            beanup,
                            'pwlf_interp_values',
                            flow_regime,
                            f_field_II,
                            f_metric_II,
                            f_field_int,
                            f_metric_int,
                            p_breakover_field,
                            p_breakover_metric,
                            q_breakover_field,
                            q_breakover_metric
                        ])
                        #
                except:
                    pass # if we have a function fail, pass over this beanup
                # (end of loop over piecewise linear sections)
            # (end of check number of piecewise linear sections)
            #
            # finally, add all the collated data from this beanup to the injectivity df
            df_injectivity = pd.concat([df_injectivity,pd.DataFrame(l_dfinj_rows,columns=df_injectivity.columns)],ignore_index=True)
            #
        # (end loop over beanups for a well)
    # (end loop over all wells)
    #
    # add the initial pressures and temperatures back in to the injectivity df
    for well in df_injectivity['well'].unique():
        #
        # well data
        df_well = df[df['well']==well][['injector_period_ID','pressure_initial','stab_type','rate','temperature']]
        #
        # slice df_well by the requested stabilisation type
        df_well = df_well[df_well['stab_type'] == d_well_data[well][i_d_well_stabilisation_method]]
        #
        # build new df of injector period -> initial pressure / temperature
        l_beanup_p0 = []
        for beanup in df_well['injector_period_ID'].unique():
            #
            # get df for this beanup only
            df_beanup = df_well[df_well['injector_period_ID'] == beanup].reset_index(drop=True)
            #
            # loop over df and find (initial) pressure, temperature at min rate
            my_min_rate = df_beanup['rate'].iloc[0]
            my_initial_pressure = df_beanup['pressure_initial'].iloc[0]
            my_initial_temperature = df_beanup['temperature'].iloc[0]
            #
            for i_df,r_df in df_beanup.iterrows():
                if r_df['rate'] < my_min_rate:
                    my_initial_pressure = r_df['pressure_initial']
                    my_min_rate = r_df['rate']
                    my_initial_temperature = r_df['temperature']
                #
            #
            # store this to list
            l_beanup_p0.append([beanup,my_initial_pressure,my_initial_temperature])
        #
        # append list to new df
        df_p0 = pd.DataFrame(l_beanup_p0,columns=['injector_period_ID','pressure_initial','temperature'])
        #
        # merge dfs (clip the beanup II data set to the well so we don't inadvertently merge other wells)
        df_well_injectivity = df_injectivity[df_injectivity['well']==well].merge(df_p0,how='inner',on='injector_period_ID')
        #
        # concatenate all wells
        try:
            df_injectivity_merged = pd.concat([df_injectivity_merged,df_well_injectivity],ignore_index=True)
        except:
            df_injectivity_merged = df_well_injectivity.copy(deep=True)
        #
        print(str('Processed bean-up injectivity estimations for well: '+well+' - number processed: '+str(len(df_well_injectivity['injector_period_ID'].unique()))))
    #
    # finally (finally), go through the bean-up df and clip outliers based on rules for frac pressure, II etc.
    for well in df_injectivity_merged['well'].unique():
        #
        f_pfrac_clip = d_well_data[well][i_d_well_minfracpressure]
        f_pfrac_max_clip = d_well_data[well][i_d_well_maxPfrac]
        f_qfrac_clip = d_well_data[well][i_d_well_maxQfrac]
        f_II_clip = d_well_data[well][i_d_well_maxII]
        #
        df_well_clip = df_injectivity_merged[df_injectivity_merged['well'] == well].copy(deep=True)
        #
        # hard force frac breakover columns below clip to -1, also clip frac breakover rates (on 0-200 m3/hr for the time being ...)
        df_well_clip.loc[(df_well_clip['Qfrac_metric']<=0.0) | (df_well_clip['Qfrac_metric']>=f_qfrac_clip),['Qfrac_field','Qfrac_metric']] = -1
        df_well_clip.loc[(df_well_clip['Pfrac_metric']<f_pfrac_clip) | (df_well_clip['Pfrac_metric']>f_pfrac_max_clip),['Pfrac_field','Pfrac_metric','Qfrac_field','Qfrac_metric']] = -1
        #
        # clip all II to 0/max II to NaN (need bitwise OR for series logic ...)
        df_well_clip.loc[(df_well_clip['II_metric']<=0.0)|(df_well_clip['II_metric']>f_II_clip),['II_field','II_metric']] = np.nan
        #
        # drop the NaN rows to get rid of spurious beanups
        df_well_clip.dropna(how='any',inplace=True)
        #
        # glue the well dfs together
        try:
            df_injectivity_clipped = pd.concat([df_injectivity_clipped,df_well_clip],ignore_index=True)
        except:
            df_injectivity_clipped = df_well_clip.copy(deep=True)
    #
    print(str('Clipped II outliers. DF length reduced from '+str(len(df_injectivity_merged))+' to '+str(len(df_injectivity_clipped))+'.'))
    print(str('Completed bean-up II calculations.'))
    #
    # return the injectivity df
    return df_injectivity_clipped

In [ ]:
def beanup_II_data(PRES_data_all, Process_WI_data, beanup_II_tracker, merged_beanup_clusters, steady_state_II_data, beanup_QC):
    #
    # Transform to add extra detail to the beanup II calculated, such as water quality and far-field reservoir pressure
    #
    # get the data sets (make sure to sort by datetime for the interpolation in this transform)
    # (also remane the time column to be consistent)
    #
    # bean-up injectivity / frac breakover data
    df_buII = beanup_II_tracker.copy(deep=True)
    # water quality data
    df_wiquality = Process_WI_data.copy(deep=True).rename({'time':'datetime'},axis='columns').sort_values(by='datetime')
    # reservoir pressure data (from RDOL tracker)
    df_PRES = PRES_data_all.copy(deep=True).rename({'time':'datetime'},axis='columns').sort_values(by='datetime')
    # bean-up datetimes from PQ cluster data
    df_bu_datetimes = merged_beanup_clusters.copy(deep=True)[['datetime','well','injector_period_ID']] # we'll use this to map bean-up injector period ID to datetime
    # steady-state II data (we'll pull the near-field PRES data from this)
    df_ss_data = steady_state_II_data.copy(deep=True)[['well','injector_period_ID','near_field_PRES_bar','shutin_length_d']]
    # bean-up QC data 
    df_bu_qc = beanup_QC.copy(deep=True)
    #
    # set up the WI quality df, don't need all the rows
    df_wiquality = df_wiquality[['datetime','sw_blend','oiw_diluted','o2_diluted']]
    #
    # do the same for the PRES data
    df_PRES = df_PRES[['Well','datetime','Datum_PRES_psi_corrected']]
    #
    # loop over each well and pull various bits of data together to apply metadata to the bean up
    for well in df_buII['well'].unique():
        #
        # get the bean-up dates for this well
        df_bu_dates = df_bu_datetimes[df_bu_datetimes['well']==well].drop(['well'],axis='columns')
        #
        # go through beanups and derive dates, lengths of each bean up
        l_beanup_data = []
        for beanup in df_bu_dates['injector_period_ID'].unique():
            #
            # slice the df to get the beanup date times only
            df_beanup = df_bu_dates[df_bu_dates['injector_period_ID'] == beanup].sort_values(by='datetime')
            #
            # take first datetime as start point and take range as the length; convert to mins
            l_beanup_data.append([beanup,df_beanup['datetime'].iloc[0],(df_beanup['datetime'].iloc[-1] - df_beanup['datetime'].iloc[0]).total_seconds() / 60.0])
            #
        #
        # build new df with these data for merging into the main df
        df_beanup_data = pd.DataFrame(l_beanup_data,columns=['injector_period_ID','datetime','beanup_length_min'])
        #
        # pull the bean-up data, don't drop any of the columns; sort by injector period ID should make it chronological
        df_buII_well = df_buII[df_buII['well']==well].sort_values(by='injector_period_ID')
        #
        # add datetime column to the bean-up data set and sort by this, shouldnt' make a difference 
        df_buII_well = df_buII_well.merge(df_beanup_data,how='left',on='injector_period_ID').sort_values(by='datetime')
        #
        # add the PWRI data using global interpolation function
        df_buII_well.loc[:,'sw_blend'] = interpolate_df_col(df_buII_well['datetime'],df_wiquality['datetime'],df_wiquality['sw_blend'],'previous')
        df_buII_well.loc[:,'oiw_diluted'] = interpolate_df_col(df_buII_well['datetime'],df_wiquality['datetime'],df_wiquality['oiw_diluted'],'previous')
        df_buII_well.loc[:,'o2_diluted'] = interpolate_df_col(df_buII_well['datetime'],df_wiquality['datetime'],df_wiquality['o2_diluted'],'previous')
        #
        # add the shut-in near well data 
        l_pbu_wells = d_well_data[well][i_d_well_offset]
        #
        # slice the PRES data for these wells
        # (for now, we'll use ALL PRES values; in the future add something here for the PBU_PFO_type)
        df_pres_well = df_PRES[df_PRES['Well'].isin(l_pbu_wells)].rename({'Datum_PRES_psi_corrected':'far_field_PRES_bar'},axis='columns').drop(['Well'],axis='columns')
        #
        # convert the psi to bar
        df_pres_well.loc[:,'far_field_PRES_bar'] = df_pres_well['far_field_PRES_bar'].divide(f_bar_to_psi)
        #
        # add the far field PRES data to the data set (both previous fill and linear so you can choose your interpolation poison)
        df_buII_well.loc[:,'far_field_PRES_pad_bar'] = interpolate_df_col(df_buII_well['datetime'],df_pres_well['datetime'],df_pres_well['far_field_PRES_bar'],'previous')
        df_buII_well.loc[:,'far_field_PRES_linear_bar'] = interpolate_df_col(df_buII_well['datetime'],df_pres_well['datetime'],df_pres_well['far_field_PRES_bar'],'linear')
        #
        # add the QC score for the bean-up
        df_well_qc = df_bu_qc[df_bu_qc['well']==well].drop(['well'],axis='columns').sort_values(by='injector_period_ID')
        df_buII_well = df_buII_well.merge(df_well_qc,how='left',on='injector_period_ID').sort_values(by='datetime')
        #
        # finally, add in near-well reservoir pressure and shut-in length
        df_ssII_well = df_ss_data[df_ss_data['well']==well].drop(['well'],axis='columns').sort_values(by='injector_period_ID')
        #
        # build list of beanup data, note st-st period must be preceded by bean-up
        l_ss_beanup_map = []
        for beanup in df_ssII_well['injector_period_ID'].unique():
            #
            # get beanup df (easier to grab values, near-field PRES / time should be same for all rows of beanup)
            df_beanup = df_ssII_well[df_ssII_well['injector_period_ID'] == beanup]
            #
            # add mapping function to list
            l_ss_beanup_map.append([beanup-1,df_beanup['near_field_PRES_bar'].iloc[0],df_beanup['shutin_length_d'].iloc[0]])
        #
        # generate df to merge
        df_ss_well_data = pd.DataFrame(l_ss_beanup_map,columns=['injector_period_ID','near_field_PRES_bar','shutin_length_d'])
        #
        # merge near-field pres / shut-in length data with bean-up df
        df_buII_well = df_buII_well.merge(df_ss_well_data,how='left',on='injector_period_ID').sort_values(by='datetime')
        #
        # append to global df
        try:
            df_buII_well_merge = pd.concat([df_buII_well_merge,df_buII_well],ignore_index=True)
        except:
            df_buII_well_merge = df_buII_well.copy(deep=True)
        #
        print(str('Completed beanup II data union for well '+well))
    #
    return df_buII_well_merge

In [ ]:
def stst_II_plot_wells(steady_state_II_data):
    #
    df = steady_state_II_data.copy(deep=True)
    #
    plot_wells = list(d_well_data.keys)
    #
    # set up scatter plot of the steady-state data
    plt.subplots(figsize=(15,7))
    #
    plt_colour_list = ['blue','mediumturquoise','darkgreen','darkviolet','darkgoldenrod','grey','gold','darkturquoise','darkgoldenrod','limegreen','saddlebrown','black','deeppink','greenyellow','darkmagenta']
    #
    for i,well in enumerate(plot_wells):
        #
        # slice df for well
        df_ststIIplot = df[df['well']==well].copy(deep=True).sort_values(by='datetime')
        #
        i_p = 0
        l_stst_II = []
        l_stst_dt = []
        #
        for period in df_ststIIplot['injector_period_ID'].unique():
            df_period = df_ststIIplot[df_ststIIplot['injector_period_ID']==period].reset_index(drop=True)
            #
            # get the middle period (we'll use this to plot the main II for this interval)
            i_cut = (int(np.nanmean(df_period.index.tolist())))
            #
            # plot single value II point for the steady-state time
            if i_p == 0:
                plt.scatter(df_period['datetime'].iloc[i_cut],df_period['II_field'].iloc[i_cut],c=plt_colour_list[i],label=well)
                i_p = 1
            else:
                plt.scatter(df_period['datetime'].iloc[i_cut],df_period['II_field'].iloc[i_cut],c=plt_colour_list[i],label='_nolabel_')
            #
            # rolling average plotter - change as necessary
            if (df_period['II_field'].iloc[i_cut] > 0.0) and (df_period['II_field'].iloc[i_cut] < 30.0):
                l_stst_II.append(df_period['II_field'].iloc[i_cut])
                l_stst_dt.append(df_period['datetime'].iloc[i_cut])
            stst_rolling = pd.DataFrame()
            stst_rolling.loc[:,'datetime'] = l_stst_dt
            stst_rolling.loc[:,'II_field'] = l_stst_II
            plt.plot(stst_rolling['datetime'],stst_rolling['II_field'].rolling(3,center=True).mean(),color=plt_colour_list[i],alpha=0.5,label='_nolabel_')
            #
            # plot ALL steady-state II with alpha blend
            plt.plot(df_period['datetime'],df_period['II_field'],color=plt_colour_list[i],alpha=0.15,label='_nolabel_')
        #
    # Plot options
    plt.ylabel('Steady-state injectivity (stb/d/psi)')
    plt.legend()
    # plt.ylim([0,100])
    plt.ylim([0,30])
    plt.show()

In [ ]:
def plot_beanup_II(beanup_II_data):
    #
    # well to plot
    plot_well = 'AW11'
    #
    df = beanup_II_data.copy(deep=True)
    df_plot = df[df['well']==plot_well]
    #
    plt.subplots(figsize=(15,15))
    #
    # colours per regime
    plot_colour = ['blue','red','green','orange']
    #
    # scatter points per measure
    plot_marker = ['|','o','x']
    #
    # bean-up II
    plt.subplot(311)
    for i_colour,flow_regime in enumerate(df_plot['flow_regime'].unique()):
        df_flow = df_plot[df_plot['flow_regime'] == flow_regime]
        for i_marker,II_measure in enumerate(df_flow['II_measure'].unique()):
            # ignore -1 values
            df_measure = df_flow[df_flow['II_measure'] == II_measure]
            plt.scatter(df_measure['injector_period_ID'],df_measure['II_field'],label=str(str(flow_regime)+' '+str(II_measure)),color=plot_colour[i_colour],marker=plot_marker[i_marker])
    #
    plt.legend()
    plt.xlabel('Injection period ID')
    plt.ylabel('Measured II (stb/d/psi)')
    plt.ylim(0,100)
    #
    # frac pressure
    plt.subplot(312)
    for i_colour,flow_regime in enumerate(df_plot['flow_regime'].unique()):
        df_flow = df_plot[df_plot['flow_regime'] == flow_regime]
        for i_marker,II_measure in enumerate(df_flow['II_measure'].unique()):
            df_measure = df_flow[df_flow['II_measure'] == II_measure]
            # ignore -1 values
            df_measure = df_measure[df_measure['Pfrac_metric']>0]
            plt.scatter(df_measure['injector_period_ID'],df_measure['Pfrac_metric'],label=str(str(flow_regime)+' '+str(II_measure)),color=plot_colour[i_colour],marker=plot_marker[i_marker])
    plt.legend(loc='lower right')
    plt.xlabel('Injection period ID')
    plt.ylabel('Measured frac breakover (barg)')
    #
    # frac rate
    plt.subplot(313)
    for i_colour,flow_regime in enumerate(df_plot['flow_regime'].unique()):
        df_flow = df_plot[df_plot['flow_regime'] == flow_regime]
        for i_marker,II_measure in enumerate(df_flow['II_measure'].unique()):
            df_measure = df_flow[df_flow['II_measure'] == II_measure]
            # ignore -1 values
            df_measure = df_measure[df_measure['Qfrac_metric']>0]
            plt.scatter(df_measure['injector_period_ID'],df_measure['Qfrac_metric'],label=str(str(flow_regime)+' '+str(II_measure)),color=plot_colour[i_colour],marker=plot_marker[i_marker])
    plt.legend(loc='lower right')
    plt.xlabel('Injection period ID')
    plt.ylabel('Measured frac breakover rate (m3/hr)')    
    #plt.ylim(100,400)
    #
    plt.show()

In [ ]:
def beanup_Pfrac_distribution(beanup_II_data):
    #
    plot_well = list(d_well_data.keys)
    #
    df_plot = beanup_II_data.copy(deep=True)
    df_plot = df_plot[(df_plot['well']==plot_well)&(df_plot['Pfrac_metric']>=d_well_data[plot_well][i_d_well_minfracpressure])]
    #
    fig,ax = plt.subplots(figsize=(15,5))
    #
    # plot the data histogram pdf
    ax.hist(df_plot['Pfrac_metric'],bins=20,density=True)
    plt.xlabel('Observed fracture breakover pressure (barg)')
    ax.set_ylabel('PDF')
    #
    # plot the data histogram cdf
    ax1 = ax.twinx()
    ax1.hist(df_plot['Pfrac_metric'],bins=20,density=True,cumulative=True,histtype='step',color='black')
    ax1.set_ylabel('CDF')
    #
    plt.show()